## Data gathering

In [1]:
from datetime import datetime, timedelta  
import pandas as pd

wildfire_data = pd.read_csv('Task_FireDetection_noRS.csv')
posData = wildfire_data[wildfire_data["isFire"]==1]
posData['DetectionDate-first'] = pd.to_datetime(posData['DetectionDate-first'], dayfirst=True).dt.strftime('%Y-%m-%d')

negData = pd.read_csv('Task_FireDetection_noRS_negClass_datesShifted_v2.csv')
negData = negData.assign(isFire = 0)
negData = negData.rename(columns={'DetectionDate_first':'DetectionDate-first','DetectionDate_last':'DetectionDate-last'})
negData['DetectionDate-first'] = pd.to_datetime(negData['DetectionDate-first'], dayfirst=True).dt.strftime('%Y-%m-%d')
negData

wildfire_data = pd.concat([posData,negData],axis=0)
wildfire_data.to_csv('Task_FireDetection_bothClasses_noRS.csv')
wildfire_data

/tmp/ipykernel_121945/3549085852.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posData['DetectionDate-first'] = pd.to_datetime(posData['DetectionDate-first'], dayfirst=True).dt.strftime('%Y-%m-%d')


,ClusterID,DetectionDate-first,DetectionDate-last,Duration,avgLatitude,avgLongitude,totalArea_km2,maxBrightness,maxBright_t31,maxFRP,avgConfidence,isFire
0,0,2000-12-28,2000-28-12,1,62.953425,-162.816950,0.979713,389.5,276.7,214.6,100.0,1
1,1,2001-05-19,2001-19-05,1,64.785600,-147.158700,0.295989,360.7,291.4,99.2,94.0,1
2,2,2001-07-16,2001-16-07,1,64.364500,-148.539500,0.250000,317.0,282.6,23.2,94.0,1
3,3,2001-09-28,2001-28-09,1,63.993100,-146.508500,0.250000,326.1,271.4,32.9,100.0,1
4,4,2002-05-19,2002-19-05,1,64.024000,-146.352200,0.250000,328.0,278.5,31.8,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2126,4258,2022-05-04,07-Jan-2022,0,55.417900,-161.893500,0.000000,0.0,0.0,0.0,0.0,0
2127,4259,2022-06-06,10-Feb-2022,0,55.422600,-161.890900,0.000000,0.0,0.0,0.0,0.0,0
2128,4260,2022-06-03,04-Feb-2022,0,55.420400,-161.875700,0.000000,0.0,0.0,0.0,0.0,0
2129,4261,2022-07-25,03-Apr-2022,0,55.418850,-161.887600,0.000000,0.0,0.0,0.0,0.0,0


In [2]:
import ee
import pandas as pd

# Trigger the authentication flow.
ee.Authenticate()


Enter verification code: 4/1AfJohXnqF3d4xsIxoL-BVH2-mxiv4Qbb3fjhZkqrDDv0J0iFNVEopcPdGXA

Successfully saved authorization token.


In [2]:
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_time_series(collection, start_date, end_date, feature_name, poi):
    """
    Retrieve time series data for the specified period.
    """
    def extract_data(image):
        value = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=350).get(feature_name)
        date = image.date().format()
        return ee.Feature(None, {'date': date, feature_name: value})

    time_series = collection.filterDate(start_date, end_date).map(extract_data)

    # Get the data as a list of features
    data_list = time_series.getInfo()['features']
    
    # Initialize lists for dates and values
    dates = []
    values = []

    # Process the list to extract dates and values
    for data in data_list:
        properties = data['properties']
        if feature_name in properties and properties[feature_name] is not None:
            dates.append(properties['date'])
            values.append(properties[feature_name])

    return dates, values

# Define datasets
datasets = {
#     'NDVI': ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI'),
#     'EVI': ee.ImageCollection('MODIS/006/MOD13Q1').select('EVI'),

#     'sur_refl_b01': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b01'),
#     'sur_refl_b02': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b02'),
#     'sur_refl_b03': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b03'),
#     'sur_refl_b07': ee.ImageCollection('MODIS/006/MOD13Q1').select('sur_refl_b07'),

#     'dayl': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('dayl'),
#     'prcp': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('prcp'),
#     'srad': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('srad'),
#     'tmax': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('tmax'),
#     'tmin': ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('tmin'),

#     'LST_Day_1km': ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km'),
#     'LST_Night_1km': ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Night_1km'),
#     'Emis_31': ee.ImageCollection('MODIS/006/MOD11A1').select('Emis_31'),
#     'Emis_32': ee.ImageCollection('MODIS/006/MOD11A1').select('Emis_32'),
#     'Clear_day_cov': ee.ImageCollection('MODIS/006/MOD11A1').select('Clear_day_cov'),
#     'Clear_night_cov': ee.ImageCollection('MODIS/006/MOD11A1').select('Clear_night_cov'),

#     'Albedo_BSA_nir': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_BSA_nir'),
#     'Albedo_BSA_shortwave': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_BSA_shortwave'),
#     'Albedo_WSA_nir': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_WSA_nir'),
#     'Albedo_WSA_shortwave': ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_WSA_shortwave'),

#     'NDSI_Snow_Cover': ee.ImageCollection('MODIS/061/MOD10A1').select('NDSI_Snow_Cover'),
#     'Nadir_Reflectance_Band1': ee.ImageCollection('MODIS/006/MCD43A4').select('Nadir_Reflectance_Band1'),

#     'Fpar': ee.ImageCollection('MODIS/061/MCD15A3H').select('Fpar'),
}

# Load your dataset
# Replace with the path to your dataset file
df = pd.read_csv('Task_FireDetection_noRS.csv')
df['DetectionDate-first'] = pd.to_datetime(df['DetectionDate-first'], format='%Y-%d-%m').dt.strftime('%Y-%m-%d')

# Process each row in the dataset
# Process and save each feature's time series
for feature_name, collection in datasets.items():
    print(f'{feature_name}', end=',')
    all_dates = []
    all_values = []

    for index, row in df.iterrows():
#         print(f'{index}', end=',')
        lat, lon, fire_date_str = row['avgLatitude'], row['avgLongitude'], row['DetectionDate-first']
        fire_date = pd.to_datetime(fire_date_str)
        poi = ee.Geometry.Point(lon, lat)
        start_date = fire_date - pd.DateOffset(days=90)
        end_date = fire_date + pd.DateOffset(days=1)

        # Get time series data for this feature and fire event
        dates, values = get_time_series(collection, start_date, end_date, feature_name, poi)

        # Add the fire date at the beginning of each list
        dates.insert(0, row['DetectionDate-first'])
        values.insert(0, row['DetectionDate-first'])

        # Append these lists to the respective all_dates and all_values lists
        all_dates.append(dates)
        all_values.append(values)

    # Create DataFrames from the lists of all dates and all values
    dates_df = pd.DataFrame(all_dates)
    values_df = pd.DataFrame(all_values)

    # Save the DataFrames to CSV files
    dates_df.to_csv(f'{feature_name}_dates.csv', index=False)
    values_df.to_csv(f'{feature_name}_values.csv', index=False)

LC_Type1,

KeyboardInterrupt: 

In [15]:
#ELEVATION DATA
import ee
import pandas as pd

# Initialize the Earth Engine module
ee.Initialize()

# Function to get elevation, slope, aspect, and other topographical features
def get_topographical_features(lat, lon):
    try:
        point = ee.Geometry.Point([lon, lat])
        
        # Load the elevation data
        elevation_dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM').mean()

        # Calculate slope and aspect
        slope = ee.Terrain.slope(elevation_dataset)
        aspect = ee.Terrain.aspect(elevation_dataset)

        # Reduce region to get values
        elev_value = elevation_dataset.reduceRegion(ee.Reducer.mean(), point, scale=350).get('DSM').getInfo()
        slope_value = slope.reduceRegion(ee.Reducer.mean(), point, scale=350).get('slope').getInfo()
        aspect_value = aspect.reduceRegion(ee.Reducer.mean(), point, scale=350).get('aspect').getInfo()

        return [
            elev_value if elev_value is not None else pd.NA,
            slope_value if slope_value is not None else pd.NA,
            aspect_value if aspect_value is not None else pd.NA
        ]
    except Exception as e:
        return [pd.NA, pd.NA, pd.NA, str(e)]  # Return NA for all values and the error message

# Load your DataFrame
df = pd.read_csv('Task_FireDetection_bothClasses_noRS.csv')

# Initialize columns for elevation, slope, and aspect
df['Elevation'] = pd.NA
df['Slope'] = pd.NA
df['Aspect'] = pd.NA

# Iterate over DataFrame rows
for index, row in df.iterrows():
    lat, lon = row['avgLatitude'], row['avgLongitude']
    elev,slo,asp = get_topographical_features(lat, lon)
#     print(elev,slo,asp)
    
    # Store results in DataFrame
    df.at[index, 'Elevation'] = elev
    df.at[index, 'Slope'] = slo
    df.at[index, 'Aspect'] = asp
    
df.to_csv('Elevation_Slope_Aspect_values.csv', index=False)
print('Gathered Elevation, Slope and Aspect!')

Gathered Elevation, Slope and Aspect!


In [29]:
# #POPULATION DATA
# import ee
# import pandas as pd

# ee.Initialize()  # Initialize the Earth Engine module.

# def get_population(lat, lon, year):
#     try:
#         point = ee.Geometry.Point([lon, lat])
#         # Assuming 'population' is the correct band name, modify as needed
#         pop_dataset = ee.ImageCollection(f"WorldPop/GP/100m/pop").select('population')
        
#         pop_value = pop_dataset.filter(ee.Filter.calendarRange(year, year, 'year')).first().reduceRegion(
#             ee.Reducer.sum(), 
#             point.buffer(1000),  # 1km buffer around the point
#             scale=500  # Scale in meters
#         ).get('population').getInfo()

#         return pop_value# if pop_value else 'No Data'
#     except Exception as e:
#         return f'Error: {str(e)}'

# # Load your dataset
# # Replace with the path to your dataset file
# df = pd.read_csv('Task_FireDetection_noRS.csv')
# df['DetectionDate-first'] = pd.to_datetime(df['DetectionDate-first'], format='%Y-%d-%m').dt.strftime('%Y-%m-%d')

# # Extract the year from the DetectionDate-first column
# df['DetectionDate-first'] = pd.to_datetime(df['DetectionDate-first'])
# df['Year'] = df['DetectionDate-first'].dt.year


# # Process each row in the dataset
# # Process and save each feature's time series
# print(f'population', end=',')
# for index, row in df.iterrows():
# #     print(f'{index}', end=',')
#     lat, lon, year = row['avgLatitude'], row['avgLongitude'], row['Year']
#     population = get_population(lat, lon, year)
#     print(f'{population}', end=',')

#     # Store results in DataFrame
#     df.at[index, 'Population'] = population
    
# df.to_csv(f'Population_values.csv', index=False)

population,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

KeyboardInterrupt: 

In [52]:
#Land Cover DATA
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_LC(lat, lon, year, lc_type):
    try:
        point = ee.Geometry.Point([lon, lat])
        # Load the MODIS land cover image collection
        LC_collection = ee.ImageCollection("MODIS/061/MCD12Q1").select(lc_type)
        # Filter the collection for the specified year
        LC_image = LC_collection.filter(ee.Filter.calendarRange(year, year, 'year')).first()
        # Reduce the region around the point
        LC_value = LC_image.reduceRegion(
            ee.Reducer.mode(), 
            point.buffer(1000),  # 1km buffer around the point
            scale=100  # Scale in meters; adjust as necessary
        ).get(lc_type).getInfo()

        return LC_value if LC_value is not None else 'No Data'
    except Exception as e:
        return f'Error: {str(e)}'

# Land cover types to process
lc_types = ['LC_Type1','LC_Type2', 'LC_Type3', 'LC_Type4', 'LC_Type5']  # 'LC_Type1', 

# Process each row in the dataset for each land cover type
for lc_type in lc_types:
    # Load your dataset
    df = pd.read_csv('Task_FireDetection_noRS.csv')
    df['DetectionDate-first'] = pd.to_datetime(df['DetectionDate-first'], format='%Y-%d-%m').dt.strftime('%Y-%m-%d')
    df['Year'] = pd.to_datetime(df['DetectionDate-first']).dt.year

    print(f'{lc_type}', end=',')
    for index, row in df.iterrows():
        lat, lon, year = row['avgLatitude'], row['avgLongitude'], row['Year']
        lc_value = get_LC(lat, lon, year, lc_type)

        # Store results in DataFrame
        df.at[index, lc_type] = lc_value

    # Save the DataFrame to a new CSV file
    df.to_csv(f'{lc_type}_values.csv', index=False)

LC_Type2,LC_Type3,LC_Type4,LC_Type5,

In [2]:
import ee
import pandas as pd

ee.Initialize()  # Initialize the Earth Engine module.

def get_time_series(collection, start_date, end_date, feature_name, poi):
    """
    Retrieve time series data for the specified period.
    """
    def extract_data(image):
        value = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=350).get(feature_name)
        date = image.date().format()
        return ee.Feature(None, {'date': date, feature_name: value})

    time_series = collection.filterDate(start_date, end_date).map(extract_data)

    # Get the data as a list of features
    data_list = time_series.getInfo()['features']
    
    # Initialize lists for dates and values
    dates = []
    values = []

    # Process the list to extract dates and values
    for data in data_list:
        properties = data['properties']
        if feature_name in properties and properties[feature_name] is not None:
            dates.append(properties['date'])
            values.append(properties[feature_name])

    return dates, values

# Define datasets
datasets = {
#     'v_component_of_wind_10m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('v_component_of_wind_10m'),
#     'u_component_of_wind_10m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('u_component_of_wind_10m'),
#     'surface_pressure': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('surface_pressure'),
    'runoff_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('runoff_sum'),
    'total_evaporation_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('total_evaporation_sum'),
    'snowfall_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snowfall_sum'),
    'snowmelt_sum': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snowmelt_sum'),
    'snow_depth': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_depth'),
    'snow_density': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_density'),
    'snow_cover': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('snow_cover'),
    'dewpoint_temperature_2m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('dewpoint_temperature_2m'),
    'temperature_2m': ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select('temperature_2m')
}

# Load your dataset
# Replace with the path to your dataset file
df = pd.read_csv('Task_FireDetection_bothClasses_noRS.csv')

# Process each row in the dataset
# Process and save each feature's time series
for feature_name, collection in datasets.items():
    print(f'{feature_name}', end=',')
    all_dates = []
    all_values = []

    for index, row in df.iterrows():
#         print(f'{index}', end=',')
        lat, lon, fire_date_str = row['avgLatitude'], row['avgLongitude'], row['DetectionDate-first']
        fire_date = pd.to_datetime(fire_date_str)
        poi = ee.Geometry.Point(lon, lat)
        start_date = fire_date - pd.DateOffset(days=90)
        end_date = fire_date + pd.DateOffset(days=1)

        # Get time series data for this feature and fire event
        dates, values = get_time_series(collection, start_date, end_date, feature_name, poi)

        # Add the fire date at the beginning of each list
        dates.insert(0, row['DetectionDate-first'])
        values.insert(0, row['DetectionDate-first'])

        # Append these lists to the respective all_dates and all_values lists
        all_dates.append(dates)
        all_values.append(values)
        
#         print(values)
    # Create DataFrames from the lists of all dates and all values
    dates_df = pd.DataFrame(all_dates)
    values_df = pd.DataFrame(all_values)

    # Save the DataFrames to CSV files
    dates_df.to_csv(f'{feature_name}_dates.csv', index=False)
    values_df.to_csv(f'{feature_name}_values.csv', index=False)

runoff_sum,total_evaporation_sum,snowfall_sum,snowmelt_sum,snow_depth,snow_density,snow_cover,dewpoint_temperature_2m,temperature_2m,

In [4]:
import ee
import pandas as pd

# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Function to extract MERRA-2 data for a given location and date
def extract_merra2_data(lat, lon, date):
    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)

    # Define the date range for pre and post wildfire analysis
    prefire_start_date = ee.Date(date).advance(-1,'year')
    prefire_end_date = ee.Date(date).advance(-1,'year').advance(1, 'month')
    postfire_start_date = ee.Date(date).advance(1,'year')
    postfire_end_date = ee.Date(date).advance(1,'year').advance(1, 'month')

    # Load the MERRA-2 dataset for the relevant features
    # Example: Surface Air Temperature
#     temp_dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
    ndvi_dataset = ee.ImageCollection('MODIS/006/MOD13Q1')
#     burned_area_dataset = ee.ImageCollection('MODIS/006/MCD64A1')
#     band_names = burned_area_dataset.first().bandNames().getInfo()
#     print("Band names:", band_names)
#     lst_dataset = ee.ImageCollection('MODIS/006/MOD11A1')
#     fire_dataset = ee.ImageCollection('MODIS/006/MOD14A1')
#     viirs_fire = ee.ImageCollection('NOAA/VIIRS/001/VNP14A1')
#     landsat8_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
#     aod_dataset = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')
    
    # Select the relevant bands 
    ndvi = ndvi_dataset.select('NDVI')
#     burned_area = burned_area_dataset.select('BurnDate').mean()
#     lst = lst_dataset.select('LST_Day_1km').mean()
#     fire = fire_dataset.select('MaxFRP')
#     fire = viirs_fire.select('MaxFRP')
    # Filter the dataset for the pre and post date ranges
#     pre_temp = temp_dataset.filterDate(prefire_start_date, prefire_end_date).mean()
#     post_temp = temp_dataset.filterDate(postfire_start_date, postfire_end_date).mean()
#     B5 = landsat8_dataset.select('B5')
#     B7 = landsat8_dataset.select('B7')

#     aod = aod_dataset.select('Optical_Depth_055')
    
    # Select the relevant bands and compute mean over the periods
    ndvi_prefire = ndvi.filterDate(prefire_start_date, prefire_end_date).mean()
    ndvi_postfire = ndvi.filterDate(postfire_start_date, postfire_end_date).mean()

#     burned_area_prefire = burned_area_dataset.filterDate(prefire_start_date, prefire_end_date).mean()
#     burned_area_postfire = burned_area_dataset.filterDate(postfire_start_date, postfire_end_date).mean()

#     lst_prefire = lst_dataset.filterDate(prefire_start_date, prefire_end_date).mean()
#     lst_postfire = lst_dataset.filterDate(postfire_start_date, postfire_end_date).mean()

#     fire_prefire = fire_dataset.filterDate(prefire_start_date, prefire_end_date).mean()
#     fire_postfire = fire.filterDate(postfire_start_date, postfire_end_date).max()

#     B5_prefire = B5.filterDate(prefire_start_date, prefire_end_date).mean()
#     B5_postfire = B5.filterDate(postfire_start_date, postfire_end_date).mean()
#     B7_prefire = B7.filterDate(prefire_start_date, prefire_end_date).mean()
#     B7_postfire = B7.filterDate(postfire_start_date, postfire_end_date).mean()

#     AOD_prefire = aod.filterDate(prefire_start_date, prefire_end_date).mean()
#     AOD_postfire = aod.filterDate(postfire_start_date, postfire_end_date).mean()
    
    # Define the scale for reduceRegion (in meters)
    scale = 375  # Example scale; adjust based on the specific dataset resolution

    # Extract the data for the point of interest with specified scale
#     pre_temp_info = pre_temp.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     post_temp_info = post_temp.reduceRegion(ee.Reducer.first(), point, scale).getInfo()

    ndvi_prefire_info = ndvi_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
    ndvi_postfire_info = ndvi_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
    
#     burned_area_prefire_info = burned_area_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     burned_area_postfire_info = burned_area_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()

#     lst_prefire_info = lst_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     lst_postfire_info = lst_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()

#     fire_prefire_info = fire_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     fire_postfire_info = fire_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()

#     B5_prefire_info = B5_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     B5_postfire_info = B5_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
    
#     B7_prefire_info = B7_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     B7_postfire_info = B7_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()

#     AOD_prefire_info = AOD_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     AOD_postfire_info = AOD_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
                
    return ndvi_prefire_info, ndvi_postfire_info#, AOD_prefire_info, AOD_postfire_info #B5_prefire_info, B5_postfire_info, B7_prefire_info, B7_postfire_info, 
#         'Pre-BurnArea': burned_area_prefire_info,
#         'Pos-BurnArea': burned_area_postfire_info,
#         'Pre-LandSurfTemp': lst_prefire_info,
#         'Pos-LandSurfTemp': lst_postfire_info,
#         'Pre-Fire': fire_prefire_info,
#         'Pos-Fire': fire_postfire_info


# Example: Load your wildfire dataset (ensure it has 'latitude', 'longitude', and 'date' columns)
wildfire_data = pd.read_csv('fire_archive_M-C61_402012.csv')
wildfire_data = wildfire_data.reindex(columns=['acq_date','confidence','latitude','longitude','brightness','bright_t31','frp'])

# Convert date strings to ee.Date objects
wildfire_data['acq_date'] = pd.to_datetime(wildfire_data['acq_date']).dt.strftime('%Y-%m-%d').apply(lambda x: ee.Date(x))

# Loop through each row in the DataFrame
print('Index','acq_date','confidence','latitude','longitude','brightness','bright_t31','frp','deltaNDVI1', 'deltaNDVI2')#, ,'Pre-NDVI1', 'Post-NDVI1', 'deltaNDVI1' 
for index, row in wildfire_data.iterrows():
    if index > 10999:
        print(index, end=', ')
        print(row.values[0].format('YYYY-dd-MM').getInfo(), end=', ')
        print(row.values[1], end=', ')
        print(row.values[2], end=', ')
        print(row.values[3], end=', ')
        print(row.values[4], end=', ')
        print(row.values[5], end=', ')
        print(row.values[6], end=', ')

        # Extract data for the current row
        result = extract_merra2_data(row['latitude'], row['longitude'], row['acq_date'])
    #     print(extract_all_values_with_placeholder(result), end=', ')
        print(result[1]['NDVI'] - result[0]['NDVI'])


Index acq_date confidence latitude longitude brightness bright_t31 frp deltaNDVI1 deltaNDVI2
11000, 2004-21-06, 53, 67.0268, -145.2309, 322.2, 299.7, 19.6, -2042.5
11001, 2004-21-06, 100, 67.018, -145.218, 365.1, 306.9, 133.5, -1821.0
11002, 2004-21-06, 94, 67.0237, -145.1921, 345.3, 297.3, 65.2, -1744.0
11003, 2004-21-06, 93, 67.003, -145.2953, 345.2, 303.4, 60.0, -2126.0
11004, 2004-21-06, 76, 67.0088, -145.2692, 335.7, 306.9, 34.2, -1873.0
11005, 2004-21-06, 11, 67.0145, -145.2432, 327.6, 306.6, 19.8, -1689.5
11006, 2004-21-06, 100, 67.0202, -145.2172, 385.5, 306.8, 241.8, -1895
11007, 2004-21-06, 92, 67.0259, -145.1913, 342.2, 296.5, 56.8, -1744.0
11008, 2004-21-06, 26, 66.9591, -145.3348, 323.1, 300.0, 18.1, -2017
11009, 2004-21-06, 66, 66.9879, -145.2046, 331.8, 300.6, 30.9, -1536.5
11010, 2004-21-06, 98, 66.9936, -145.1786, 357.0, 298.7, 101.2, -1473.0
11011, 2004-21-06, 94, 66.956, -145.2958, 347.5, 303.1, 70.4, -2035.0
11012, 2004-21-06, 14, 66.9617, -145.2698, 328.7, 301.6, 2

11115, 2004-21-06, 89, 64.8751, -141.9949, 336.9, 300.5, 119.7, -867.5
11116, 2004-21-06, 72, 64.8788, -141.8866, 326.6, 296.5, 71.3, -3256.5
11117, 2004-21-06, 100, 66.98, -145.1664, 437.6, 307.5, 1603.6, -1779.5
11118, 2004-21-06, 100, 66.9776, -145.2138, 409.6, 307.9, 875.3, -1783.0
11119, 2004-21-06, 100, 66.9662, -145.2187, 365.5, 289.7, 262.5, -1312.5
11120, 2004-21-06, 100, 66.9926, -145.1703, 385.8, 301.5, 486.5, -2429
11121, 2004-21-06, 92, 66.964, -145.2659, 352.0, 293.5, 154.1, -1610
11122, 2004-21-06, 87, 66.9832, -145.1275, 375.0, 283.2, 361.0, -1381
11123, 2004-21-06, 93, 64.8927, -141.8871, 344.5, 299.0, 170.3, -1035.0
11124, 2004-21-06, 52, 64.8662, -142.0029, 319.5, 297.3, 40.8, -479
11125, 2004-21-06, 93, 64.8835, -141.8921, 344.2, 297.7, 169.7, -3374
11126, 2004-21-06, 67, 64.8801, -141.9989, 329.0, 301.1, 77.4, -2261.5
11127, 2004-21-06, 87, 64.8974, -141.891, 338.7, 298.8, 128.4, -1376.0
11128, 2004-21-06, 90, 64.9094, -141.9478, 338.0, 300.7, 124.8, -1059.5
11129,

11347, 2004-22-06, 100, 65.201, -141.5738, 323.7, 289.0, 22.7, -1910.0
11348, 2004-22-06, 76, 65.4432, -143.1103, 316.9, 283.2, 14.2, -1349.5
11349, 2004-22-06, 89, 65.44, -143.0899, 321.9, 284.0, 19.8, -1948.5
11350, 2004-22-06, 28, 65.1925, -141.5816, 303.3, 288.3, 6.3, -1538.5
11351, 2004-22-06, 10, 65.4334, -143.1033, 300.2, 284.7, 4.8, -1685.5
11352, 2004-22-06, 49, 65.1832, -141.5742, 304.8, 289.5, 8.2, -2324.5
11353, 2004-22-06, 80, 64.9306, -141.9268, 310.4, 288.3, 11.6, -2065.5
11354, 2004-22-06, 100, 64.9271, -141.9064, 328.9, 290.8, 30.2, -2831.5
11355, 2004-22-06, 81, 64.9236, -141.8857, 310.6, 288.0, 12.0, -2607
11356, 2004-22-06, 49, 64.9289, -141.9751, 302.4, 286.6, 7.0, -1701.0
11357, 2004-22-06, 100, 65.9263, -149.3024, 351.7, 294.6, 81.5, -1460.5
11358, 2004-22-06, 91, 65.9236, -149.2775, 323.1, 290.6, 25.1, -1882.0
11359, 2004-22-06, 61, 65.9169, -149.3081, 309.8, 288.7, 11.9, -1876
11360, 2004-22-06, 62, 65.8964, -149.5969, 311.8, 287.3, 16.1, -2367.5
11361, 2004-22

11579, 2004-22-06, 65, 67.0375, -145.1557, 337.6, 304.2, 32.7, -853
11580, 2004-22-06, 68, 67.029, -145.1105, 340.1, 309.4, 38.5, -1808.5
11581, 2004-22-06, 95, 67.0247, -145.088, 355.9, 307.1, 73.1, -1575.5
11582, 2004-22-06, 96, 67.0204, -145.0654, 372.0, 308.4, 123.8, -1575.5
11583, 2004-22-06, 100, 67.0161, -145.0428, 390.3, 313.9, 212.9, -3341
11584, 2004-22-06, 100, 67.0118, -145.0202, 372.4, 308.0, 129.6, -741.0
11585, 2004-22-06, 91, 67.2259, -146.2368, 352.8, 301.8, 59.6, -2188
11586, 2004-22-06, 99, 67.214, -146.1701, 357.4, 300.0, 72.7, -939
11587, 2004-22-06, 94, 67.21, -146.1479, 370.5, 296.8, 115.7, -1152.5
11588, 2004-22-06, 23, 67.029, -145.166, 334.2, 307.2, 24.3, -932.5
11589, 2004-22-06, 74, 67.0247, -145.1435, 337.7, 304.7, 27.8, -2008.5
11590, 2004-22-06, 95, 67.0162, -145.0984, 359.9, 308.4, 81.0, -2598.5
11591, 2004-22-06, 100, 67.0119, -145.0758, 399.4, 316.1, 257.1, -2563.0
11592, 2004-22-06, 68, 67.0076, -145.0532, 350.6, 315.3, 53.4, -2694.5
11593, 2004-22-06

11811, 2004-22-06, 76, 67.0412, -145.1325, 326.7, 295.9, 19.7, -1401.5
11812, 2004-22-06, 66, 67.0351, -145.1769, 333.5, 300.6, 24.4, -1408
11813, 2004-22-06, 100, 67.0289, -145.2213, 433.0, 322.7, 510.8, -2733.5
11814, 2004-22-06, 100, 67.0259, -145.2434, 435.8, 340.1, 539.1, -2149.0
11815, 2004-22-06, 95, 67.0228, -145.2655, 352.6, 314.5, 54.6, -2325.0
11816, 2004-22-06, 83, 67.1345, -146.2391, 345.6, 295.7, 44.0, -818.0
11817, 2004-22-06, 94, 67.1493, -146.2034, 389.9, 304.0, 190.1, -1402.5
11818, 2004-22-06, 100, 67.1462, -146.2253, 446.2, 323.0, 657.4, -2086.0
11819, 2004-22-06, 100, 67.143, -146.2473, 485.0, 334.3, 1276.3, -1374.5
11820, 2004-22-06, 94, 67.1398, -146.2692, 347.3, 295.1, 48.4, -2237.5
11821, 2004-22-06, 75, 67.161, -146.1897, 417.6, 306.7, 369.9, -1606.0
11822, 2004-22-06, 94, 67.1578, -146.2117, 401.8, 320.6, 255.7, -1673
11823, 2004-22-06, 100, 67.1547, -146.2336, 378.5, 316.1, 137.3, -1699.5
11824, 2004-22-06, 100, 67.1515, -146.2555, 369.9, 315.4, 104.3, -1888

...

14709, 2004-26-06, 79, 67.1673, -146.4621, 310.2, 286.4, 33.6, -2336.0
14710, 2004-26-06, 8, 67.0831, -145.0912, 300.1, 287.0, 13.9, -2620.5
14711, 2004-26-06, 43, 67.0546, -145.2796, 309.0, 286.5, 16.1, -3383.5
14712, 2004-26-06, 52, 67.0786, -145.0733, 309.3, 289.2, 16.1, -2019.0
14713, 2004-26-06, 98, 67.0713, -145.1095, 319.1, 290.3, 27.9, -2867.5
14714, 2004-26-06, 47, 64.5615, -145.1568, 309.4, 285.3, 13.3, -2540.5
14715, 2004-26-06, 26, 65.9559, -149.4739, 306.4, 284.8, 20.8, -1854
14716, 2004-26-06, 43, 65.1839, -141.6016, 301.6, 287.8, 6.5, -2634.5
14717, 2004-26-06, 28, 64.8899, -141.8216, 302.8, 287.2, 6.4, -2578.5
14718, 2004-26-06, 12, 64.8718, -141.7232, 305.8, 287.4, 8.1, -1806.5
14719, 2004-26-06, 23, 64.8469, -141.8343, 301.6, 287.1, 6.5, -2577.5
14720, 2004-26-06, 20, 65.1724, -145.1011, 300.2, 283.8, 8.7, -1307.0
14721, 2004-26-06, 18, 64.3398, -141.6978, 300.3, 286.6, 6.1, -2816.5
14722, 2004-26-06, 100, 64.5736, -145.1746, 339.3, 286.7, 57.7, -2349
14723, 2004-26-0

14941, 2004-26-06, 76, 67.0547, -145.2658, 335.6, 303.7, 30.0, -2278
14942, 2004-26-06, 100, 67.0505, -145.2883, 360.9, 306.9, 87.6, -3284.0
14943, 2004-26-06, 96, 67.0464, -145.3107, 350.7, 305.2, 62.1, -2865.0
14944, 2004-26-06, 78, 67.0422, -145.3331, 329.5, 303.2, 23.1, -3234.5
14945, 2004-26-06, 80, 65.9418, -149.5712, 328.1, 296.4, 34.8, -2436.5
14946, 2004-26-06, 91, 65.9356, -149.5983, 339.5, 298.9, 60.1, -1499.5
14947, 2004-26-06, 80, 65.9296, -149.625, 333.0, 297.3, 44.5, -1511
14948, 2004-26-06, 0, 66.881, -145.2357, 367.6, 306.3, 110.1, -2438.5
14949, 2004-26-06, 97, 66.9022, -145.2889, 353.6, 306.1, 64.9, -2332.5
14950, 2004-26-06, 78, 66.8981, -145.3113, 340.5, 303.5, 37.7, -2024
14951, 2004-26-06, 91, 66.9066, -145.3216, 346.2, 305.8, 49.2, -1952.5
14952, 2004-26-06, 76, 66.915, -145.3318, 337.0, 306.1, 31.5, -1580.5
14953, 2004-26-06, 67, 66.9235, -145.3421, 335.1, 306.7, 28.1, -1664
14954, 2004-26-06, 55, 66.9361, -145.3299, 325.3, 305.5, 12.6, -1666.5
14955, 2004-26-0

15174, 2004-27-06, 62, 64.0468, -141.5785, 304.9, 286.3, 8.8, -1044
15175, 2004-27-06, 16, 64.3182, -141.6504, 302.6, 288.0, 6.4, -1980
15176, 2004-27-06, 46, 64.3298, -141.6346, 302.5, 288.2, 6.6, -1681
15177, 2004-27-06, 61, 64.3273, -141.6564, 305.3, 287.8, 8.2, -1239
15178, 2004-27-06, 29, 64.384, -141.337, 300.5, 286.7, 5.6, -1289
15179, 2004-27-06, 83, 64.3932, -141.3421, 311.6, 287.1, 13.2, -1482
15180, 2004-27-06, 80, 64.3907, -141.3642, 310.4, 287.6, 12.1, -1879
15181, 2004-27-06, 20, 64.869, -141.7284, 300.5, 286.9, 5.7, -1692
15182, 2004-27-06, 86, 64.4847, -145.0231, 313.0, 289.2, 12.2, -2805
15183, 2004-27-06, 23, 64.499, -145.0566, 300.3, 287.2, 4.8, -2181
15184, 2004-27-06, 76, 64.5219, -144.9624, 309.0, 290.1, 8.9, -3326
15185, 2004-27-06, 53, 64.5049, -145.1499, 303.1, 287.3, 6.3, -1608
15186, 2004-27-06, 34, 64.5267, -145.1951, 303.5, 288.3, 6.0, -890
15187, 2004-27-06, 75, 64.5238, -145.2152, 308.6, 288.6, 9.1, -310
15188, 2004-27-06, 100, 64.5639, -145.135, 322.9, 2

15411, 2004-27-06, 38, 67.2114, -146.242, 302.5, 285.1, 5.8, -409
15412, 2004-27-06, 15, 67.2508, -146.0808, 318.7, 290.5, 16.6, 11
15413, 2004-27-06, 64, 67.1956, -146.5634, 346.8, 288.6, 50.3, -1540
15414, 2004-27-06, 87, 67.2042, -146.5715, 422.1, 313.7, 414.2, -1980
15415, 2004-27-06, 29, 66.8923, -145.5773, 311.9, 283.2, 109.2, -795
15416, 2004-27-06, 55, 65.9488, -149.5654, 303.5, 286.2, 47.5, -1097
15417, 2004-27-06, 100, 66.8592, -145.4596, 357.7, 285.0, 797.3, -1957
15418, 2004-27-06, 100, 66.8774, -145.4562, 360.9, 286.8, 863.8, -1987
15419, 2004-27-06, 100, 66.8802, -145.5679, 331.9, 284.1, 310.4, -566
15420, 2004-27-06, 100, 66.856, -145.4691, 362.2, 285.4, 905.7, -1879
15421, 2004-27-06, 100, 66.8742, -145.4659, 356.0, 286.1, 737.6, -1741
15422, 2004-27-06, 100, 66.8773, -145.5775, 326.4, 283.5, 240.6, -654
15423, 2004-27-06, 100, 66.8924, -145.4626, 362.8, 288.5, 918.8, -1406
15424, 2004-27-06, 40, 66.8955, -145.5744, 306.8, 283.1, 81.1, -1158
15425, 2004-27-06, 100, 66.9

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [ ]:
import ee
import pandas as pd
import numpy as np
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Function to extract MERRA-2 data for a given location and date
def extract_merra2_data(lat, lon, date):
    # Define the point of interest
    point = ee.Geometry.Point(lon, lat)

    # Define the date range for pre and post wildfire analysis
    prefire_start_date = ee.Date(date).advance(-1,'year')
    prefire_end_date = ee.Date(date).advance(-1,'year').advance(1, 'month')
    postfire_start_date = ee.Date(date).advance(1,'year')
    postfire_end_date = ee.Date(date).advance(1,'year').advance(1, 'month')

    prefire_start_date2 = ee.Date(date).advance(-1, 'week')
    prefire_end_date2 = ee.Date(date).advance(1, 'week')

    # Load the MERRA-2 dataset for the relevant features
    ndvi_dataset = ee.ImageCollection('MODIS/006/MOD13Q1')
    aod_dataset = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')

    # Select the relevant bands 
    ndvi = ndvi_dataset.select('NDVI')
    aod = aod_dataset.select('Optical_Depth_055')
    
    # Select the relevant bands and compute mean over the periods
    ndvi_prefire = ndvi.filterDate(prefire_start_date, prefire_end_date).mean()
    ndvi_postfire = ndvi.filterDate(postfire_start_date, postfire_end_date).mean()
    
    aod_prefire = aod.filterDate(prefire_start_date2, prefire_end_date2).max()

    # Define the scale for reduceRegion (in meters)
    scale = 375  # Example scale; adjust based on the specific dataset resolution

    # Extract the data for the point of interest with specified scale
    ndvi_prefire_info = ndvi_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
    ndvi_postfire_info = ndvi_postfire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     print(ndvi_postfire_info['NDVI']-ndvi_prefire_info['NDVI'])
    
    aod_postfire_info = aod_prefire.reduceRegion(ee.Reducer.first(), point, scale).getInfo()
#     print(ndvi_postfire_info2['NDVI']-ndvi_prefire_info2['NDVI'])
                
    return ndvi_prefire_info, ndvi_postfire_info, aod_postfire_info
#ndvi_postfire_info['NDVI']-ndvi_prefire_info['NDVI'], ndvi_postfire_info2['NDVI']-ndvi_prefire_info2['NDVI']

# Example: Load your wildfire dataset (ensure it has 'latitude', 'longitude', and 'date' columns)
wildfire_data = pd.read_csv('fire_archive_M-C61_402012.csv')
wildfire_data = wildfire_data.reindex(columns=['acq_date','confidence','latitude','longitude','brightness','bright_t31','frp'])

# Convert date strings to ee.Date objects
wildfire_data['acq_date'] = pd.to_datetime(wildfire_data['acq_date']).dt.strftime('%Y-%m-%d').apply(lambda x: ee.Date(x))

# Loop through each row in the DataFrame
print('Index','acq_date','confidence','latitude','longitude','brightness','bright_t31','frp','deltaNDVI1', 'deltaNDVI2')#, ,'Pre-NDVI1', 'Post-NDVI1', 'deltaNDVI1' 
for index, row in wildfire_data.iterrows():
    if index > 18974:
        print(index, end=', ')
        print(row.values[0].format('YYYY-dd-MM').getInfo(), end=', ')
        print(row.values[1], end=', ')
        print(row.values[2], end=', ')
        print(row.values[3], end=', ')
        print(row.values[4], end=', ')
        print(row.values[5], end=', ')
        print(row.values[6], end=', ')
        # Extract data for the current row
        result = extract_merra2_data(row['latitude'], row['longitude'], row['acq_date'])
        print(extract_all_values_with_placeholder(result))
    #     print({result[1]['NDVI']-result[0]['NDVI']},{result[3]['NDVI']-result[2]['NDVI']})

Index acq_date confidence latitude longitude brightness bright_t31 frp deltaNDVI1 deltaNDVI2
18975, 2004-29-06, 100, 65.25, -146.963, 363.8, 301.6, 112.3, 7203.5, 5409, 3117
18976, 2004-29-06, 100, 65.2546, -146.9859, 383.4, 306.5, 201.3, 7564, 5816, 3117
18977, 2004-29-06, 100, 65.259, -147.0079, 392.9, 311.1, 258.8, 7475.5, 4530.5, 3117
18978, 2004-29-06, 96, 65.2636, -147.0309, 352.6, 299.1, 78.9, 7965, 4899.5, 3117
18979, 2004-29-06, 10, 64.0295, -141.5653, 321.4, 299.4, 23.3, 6527, 5506.5, 3117
18980, 2004-29-06, 87, 64.0623, -141.6929, 332.9, 299.0, 58.6, 7257, 5407, 3117
18981, 2004-29-06, 79, 64.0695, -141.721, 328.1, 292.3, 46.9, 7448.5, 5897, 3117
18982, 2004-29-06, 100, 65.199, -146.6571, 399.0, 306.2, 304.9, 7554, 5046.5, 3117
18983, 2004-29-06, 100, 65.2036, -146.6795, 384.5, 304.2, 207.2, 7217.5, 5322, 3117
18984, 2004-29-06, 52, 65.2174, -146.7478, 328.3, 298.6, 20.3, 7279.5, 5616, 3117
18985, 2004-29-06, 87, 65.2311, -146.8152, 334.0, 300.3, 31.8, 6877.5, 5855.5, 3117
1

19172, 2004-29-06, 100, 65.1797, -141.3908, 399.5, 310.3, 480.2, 7737, 5925, 3117
19173, 2004-29-06, 100, 65.1865, -141.4185, 426.5, 318.2, 873.9, 8247.5, 5141, 3117
19174, 2004-29-06, 100, 65.1933, -141.4461, 379.1, 302.2, 279.3, 7526, 5998, 3117
19175, 2004-29-06, 69, 65.572, -143.0379, 333.9, 300.1, 41.8, 7052.5, 5644, 3117
19176, 2004-29-06, 99, 65.5777, -143.063, 359.7, 304.2, 125.1, 6941.5, 6170, 3117
19177, 2004-29-06, 96, 65.9467, -144.7363, 351.2, 302.9, 74.1, 7985, 6322, 3117
19178, 2004-29-06, 45, 65.9519, -144.7605, 328.8, 299.7, 20.8, 7273, 5508.5, 3117
19179, 2004-29-06, 97, 65.962, -144.8086, 353.0, 302.3, 78.2, 6973.5, 5932, 3117
19180, 2004-29-06, 100, 65.9672, -144.8331, 386.2, 305.5, 229.3, 7455.5, 6273, 3117
19181, 2004-29-06, 72, 65.9826, -144.9062, 326.9, 300.4, 20.3, 7283.5, 5620.5, 3117
19182, 2004-29-06, 57, 65.9877, -144.9305, 324.6, 299.3, 18.3, 7283.5, 5620.5, 3117
19183, 2004-29-06, 85, 65.9928, -144.9545, 331.9, 297.8, 32.2, 8082, 5972.5, 3117
19184, 2004-

19370, 2004-29-06, 70, 65.2809, -145.2747, 332.2, 304.6, 30.6, 6937.5, 5591.5, 3117
19371, 2004-29-06, 100, 65.2859, -145.298, 428.3, 313.9, 646.3, 6749.5, 5948.5, 3117
19372, 2004-29-06, 97, 65.2909, -145.3214, 353.2, 301.1, 88.9, 6672.5, 5943, 3117
19373, 2004-29-06, 88, 65.2898, -145.2633, 340.2, 301.5, 49.1, 7163.5, 5973.5, 3117
19374, 2004-29-06, 96, 65.2948, -145.2867, 351.9, 302.9, 82.8, 6701.5, 5810, 3117
19375, 2004-29-06, 88, 65.272, -145.128, 335.3, 303.4, 35.5, 6632.5, 5838.5, 3117
19376, 2004-29-06, 92, 65.2811, -145.117, 343.0, 305.7, 53.4, 7010.5, 6349.5, 3117
19377, 2004-29-06, 77, 64.3951, -141.3003, 339.9, 292.6, 83.5, 7394, 4272.5, 3117
19378, 2004-29-06, 100, 65.3014, -145.1587, 393.7, 309.7, 293.7, 6414, 4530.5, 3117
19379, 2004-29-06, 91, 65.3065, -145.1825, 345.3, 301.8, 61.6, 6909, 5806, 3117
19380, 2004-29-06, 69, 64.405, -141.2865, 326.1, 296.4, 42.8, 7019, 4262.5, 3117
19381, 2004-29-06, 69, 65.3152, -145.17, 328.9, 301.7, 28.4, 6750.5, 5438.5, 3117
19382, 20

19568, 2004-29-06, 100, 64.375, -141.3441, 361.1, 297.3, 90.6, 7410, 5245, 3117
19569, 2004-29-06, 100, 64.3776, -141.3218, 366.0, 298.0, 109.4, 7451, 4484, 3117
19570, 2004-29-06, 78, 64.3583, -141.4036, 322.4, 293.8, 19.1, 6475, 5534, 3117
19571, 2004-29-06, 72, 64.3608, -141.3823, 329.4, 297.6, 25.6, 6908.5, 4434, 3117
19572, 2004-29-06, 88, 64.3633, -141.3602, 351.9, 303.1, 66.2, 7180, 4815, 3117
19573, 2004-29-06, 100, 64.366, -141.3374, 364.3, 301.5, 105.0, 7134, 5290, 3117
19574, 2004-29-06, 93, 64.0325, -142.1851, 343.6, 298.0, 47.7, 6977.5, 5496, 3117
19575, 2004-29-06, 100, 64.0352, -142.1638, 427.7, 310.8, 500.2, 7258, 5617, 3117
19576, 2004-29-06, 100, 64.0927, -141.6912, 465.8, 316.5, 1045.4, 7396, 5568, 3117
19577, 2004-29-06, 100, 64.0953, -141.6696, 456.8, 316.6, 893.5, 6322, 5395.5, 3117
19578, 2004-29-06, 97, 64.0209, -142.2009, 354.5, 299.4, 73.0, 7307.5, 5444, 3117
19579, 2004-29-06, 100, 64.0236, -142.1792, 364.9, 301.1, 101.6, 7641.5, 6780, 3117
19580, 2004-29-06,

19766, 2004-29-06, 92, 67.0954, -146.6808, 343.5, 304.9, 44.3, 6308, 6974.5, 3117
19767, 2004-29-06, 88, 67.0924, -146.7031, 336.7, 301.2, 33.9, 7357.5, 7018.5, 3117
19768, 2004-29-06, 82, 67.0893, -146.7253, 328.1, 299.6, 22.4, 7473, 6604.5, 3117
19769, 2004-29-06, 56, 67.1132, -146.6221, 331.2, 300.8, 22.4, 7939.5, 6140, 2897
19770, 2004-29-06, 99, 67.1101, -146.6443, 357.5, 304.8, 74.4, 7716.5, 6034.5, 2868
19771, 2004-29-06, 100, 67.1071, -146.6665, 381.6, 307.5, 158.0, 7799.5, 6562, 3117
19772, 2004-29-06, 92, 67.104, -146.6888, 343.9, 300.4, 46.7, 7416, 7068, 3117
19773, 2004-29-06, 52, 67.1426, -146.5487, 327.4, 301.0, 15.2, 8071.5, 6423, 3117
19774, 2004-29-06, 92, 67.1395, -146.5711, 343.9, 303.4, 40.1, 7971, 6603.5, 3117
19775, 2004-29-06, 51, 67.145, -146.6016, 329.8, 301.1, 19.5, 8353, 5387.5, 3117
19776, 2004-29-06, 71, 67.1389, -146.6461, 335.3, 302.6, 29.5, 8360, 6092, 3117
19777, 2004-29-06, 83, 65.4816, -143.0557, 329.3, 297.4, 24.6, 7452.5, 5693.5, 3117
19778, 2004-29

...

25738, 2004-02-07, 100, 65.1029, -146.0225, 335.3, 288.4, 37.9, 7518.5, 5117, 3117
25739, 2004-02-07, 78, 65.1002, -146.0468, 325.2, 289.0, 22.2, 7708.5, 4628, 3117
25740, 2004-02-07, 21, 65.0975, -146.0709, 316.9, 286.7, 14.0, 8292, 6237, 3117
25741, 2004-02-07, 17, 65.1484, -145.7003, 300.2, 284.3, 6.0, 7162, 6296.5, 3117
25742, 2004-02-07, 30, 65.125, -145.9127, 317.6, 286.9, 15.0, 7166.5, 6317.5, 3117
25743, 2004-02-07, 60, 65.1225, -145.9357, 320.4, 286.4, 17.5, 7172, 6435, 3117
25744, 2004-02-07, 100, 65.1199, -145.9586, 328.0, 286.5, 28.2, 7144, 6679, 3117
25745, 2004-02-07, 100, 65.1173, -145.9818, 325.1, 285.2, 24.3, 7042, 6073, 3117
25746, 2004-02-07, 97, 65.1147, -146.0051, 318.3, 285.7, 16.3, 7629.5, 6453.5, 3117
25747, 2004-02-07, 46, 65.1121, -146.0288, 306.7, 286.9, 5.7, 7884.5, 6750, 3117
25748, 2004-02-07, 45, 65.176, -145.5385, 302.6, 285.3, 7.9, 6510.5, 6162.5, 3117
25749, 2004-02-07, 75, 65.1733, -145.5628, 308.8, 285.2, 12.9, 6571.5, 5732.5, 3117
25750, 2004-02-07,

25937, 2004-02-07, 64, 68.4538, -141.143, 308.4, 292.1, 50.4, 6797.5, 5853, 2645
25938, 2004-02-07, 56, 67.4508, -143.5775, 303.5, 291.4, 47.2, 7427.5, 6919.5, 1868
25939, 2004-02-07, 57, 67.4477, -143.5819, 303.7, 291.4, 48.2, 6959.5, 6604.5, 1979
25940, 2004-02-07, 36, 68.4286, -141.2337, 322.9, 295.7, 21.3, 6950.5, 6113.5, 2488
25941, 2004-02-07, 54, 68.4671, -141.1036, 323.7, 296.2, 22.4, 6940, 5803, 2527
25942, 2004-02-07, 39, 65.1252, -145.6373, 321.2, 293.5, 23.6, 6611.5, 6396, 3117
25943, 2004-02-07, 42, 65.1227, -145.6078, 322.6, 295.6, 25.2, 6764.5, 5737, 3117
25944, 2004-02-07, 65, 65.1115, -145.8621, 330.8, 295.6, 39.3, 7114.5, 5775.5, 3117
25945, 2004-02-07, 71, 64.554, -141.6944, 330.7, 301.0, 27.2, 7448.5, 5435.5, 3117
25946, 2004-02-07, 90, 64.5606, -141.837, 349.9, 298.7, 67.4, 7024.5, 6853, 3117
25947, 2004-02-07, 100, 64.558, -141.8141, 362.2, 304.5, 100.0, 7060, 6658, 3117
25948, 2004-02-07, 100, 64.5514, -141.8432, 471.0, 340.1, 1198.7, 7140, 3970.5, 3117
25949, 20

26037, 2004-02-07, 64, 66.8084, -145.2239, 342.2, 301.9, 66.6, 7254.5, 5321.5, 3117
26038, 2004-02-07, 48, 66.8058, -145.191, 335.5, 298.6, 49.5, 6964.5, 5720.5, 3117
26039, 2004-02-07, 0, 66.7951, -145.334, 332.7, 297.9, 40.5, 7092.5, 5053, 3117
26040, 2004-02-07, 74, 66.2392, -141.1853, 324.9, 295.9, 22.0, 7203.5, 5789.5, 3117
26041, 2004-02-07, 37, 66.2326, -141.2165, 327.7, 295.8, 22.9, 7199, 5521.5, 3117
26042, 2004-02-07, 98, 66.2298, -141.1912, 357.0, 298.3, 89.1, 7314, 5837.5, 3117
26043, 2004-02-07, 66, 66.227, -141.1659, 321.7, 297.0, 17.4, 7343.5, 6572.5, 3117
26044, 2004-02-07, 95, 66.226, -141.248, 347.9, 298.9, 61.8, 7332, 6031.5, 3117
26045, 2004-02-07, 100, 66.2232, -141.2225, 369.5, 298.3, 130.6, 7998.5, 5883, 2686
26046, 2004-02-07, 96, 66.2167, -141.2542, 350.3, 298.0, 70.3, 7955, 4984, 3117
26047, 2004-02-07, 98, 66.2139, -141.229, 357.2, 297.7, 89.6, 7955, 4984, 3117
26048, 2004-02-07, 84, 66.2255, -141.3611, 327.4, 295.9, 27.0, 7869.5, 5754, 3117
26049, 2004-02-07

26236, 2004-02-07, 100, 64.4246, -141.3045, 378.0, 302.4, 156.8, 6640.5, 3890, 3117
26237, 2004-02-07, 100, 64.4273, -141.3273, 436.1, 317.5, 615.8, 6554.5, 3671, 3117
26238, 2004-02-07, 93, 64.4155, -141.3107, 343.7, 303.8, 45.5, 6995.5, 4210, 3117
26239, 2004-02-07, 60, 64.3029, -141.6179, 321.4, 299.1, 29.1, 7147, 5730, 3117
26240, 2004-02-07, 44, 64.1049, -141.6234, 313.4, 291.9, 19.0, 7055.5, 5714, 3117
26241, 2004-02-07, 61, 64.1065, -141.6702, 323.2, 290.7, 41.5, 7370.5, 5782, 3117
26242, 2004-02-07, 59, 64.3288, -141.3118, 330.2, 300.0, 44.8, 6890, 4413.5, 3117
26243, 2004-02-07, 78, 64.2453, -141.6951, 334.2, 295.0, 64.0, 7634, 5843, 3117
26244, 2004-02-07, 87, 64.2631, -141.6781, 331.9, 296.7, 57.4, 7133, 6228, 3117
26245, 2004-02-07, 63, 64.2596, -141.6807, 328.8, 297.2, 45.3, 7238.5, 5170, 3117
26246, 2004-02-07, 44, 64.3708, -141.299, 315.2, 297.7, 14.8, 7443, 6530, 3117
26247, 2004-02-07, 90, 64.4168, -141.2747, 338.3, 298.8, 73.4, 7202, 5918, 3117
26248, 2004-02-07, 93, 

26336, 2004-02-07, 67, 64.0168, -141.6617, 320.2, 295.4, 26.2, 7069.5, 5634.5, 3117
26337, 2004-02-07, 94, 64.4282, -141.3348, 373.7, 297.0, 262.7, 6843.5, 4231.5, 3117
26338, 2004-02-07, 89, 64.4202, -141.3668, 347.3, 295.3, 110.6, 6727.5, 4779, 3117
26339, 2004-02-07, 85, 65.2692, -141.4802, 338.4, 298.2, 61.9, 6277, 5855.5, 3069
26340, 2004-02-07, 81, 65.2837, -141.5789, 344.6, 298.1, 83.2, 6555.5, 5296, 3117
26341, 2004-02-07, 58, 64.3931, -145.1999, 328.0, 298.2, 61.6, 7590, 6062, 3117
26342, 2004-02-07, 38, 64.3925, -145.1513, 316.7, 294.4, 26.5, 7609.5, 6452.5, 3054
26343, 2004-02-07, 76, 64.3918, -145.2073, 329.4, 298.3, 67.8, 7438, 5921, 3111
26344, 2004-02-07, 54, 64.5291, -144.8896, 315.4, 296.3, 25.3, 7056, 5629, 3117
26345, 2004-02-07, 51, 64.4665, -145.1629, 313.7, 294.9, 18.8, 7707.5, 5424, 2901
26346, 2004-02-07, 39, 64.456, -145.1993, 312.3, 294.5, 15.7, 7876.5, 6246.5, 1523
26347, 2004-02-07, 100, 64.4993, -145.1017, 361.7, 298.3, 252.3, 8003, 6290, 2938
26348, 2004-0

In [17]:
import ee
import pandas as pd

# Initialize the Earth Engine module.
ee.Initialize()

# Define your time range and region of interest
start_date = '2014-12-01'  # Replace with your start date
end_date = '16-12-31'    # Replace with your end date
region = ee.Geometry.Rectangle([-168, 64, -141, 71.365162])  # Replace with the coordinates of your region

# Load the MODIS thermal anomalies/fire dataset
modis_fire = ee.ImageCollection('MODIS/006/MOD14A1').filterDate(start_date, end_date).filterBounds(region)

# Function to extract fire information from each image
def extract_fire_details(image):
    # Use the 'FireMask' band to identify fire pixels (value of 9 indicates active fire)
    fire_mask = image.select('FireMask').eq(9)

    # Select the FRP band and mask it with the fire pixels
    frp = image.select('MaxFRP').updateMask(fire_mask)

    # Create an image that contains the latitude and longitude of each pixel
    latlon = ee.Image.pixelCoordinates(ee.Projection('EPSG:4326')).addBands(frp)

    # Mask the latlon image with the fire mask
    fire_latlon = latlon.updateMask(fire_mask)

    # Reduce the fire pixels to lists of coordinates and FRP
    fire_info = fire_latlon.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=region, 
        scale=1000,
        maxPixels=1e8
    )

    # Get the date of the image
    date = image.date().format('YYYY-MM-dd')
    
    # Return the fire information with the date
    return ee.Feature(None, {'date': date, 'fire_info': fire_info})

# Apply the function to each image in the collection
fire_events = modis_fire.map(extract_fire_details)

# Gather the results
fire_events_info = fire_events.getInfo()['features']

# Extracting and printing fire details
for event in fire_events_info:
    properties = event['properties']
    fire_info = properties['fire_info']
    date = properties['date']

    if fire_info['MaxFRP']:  # Check if there are any fire data
        lats = fire_info['latitude']
        lons = fire_info['longitude']
        frps = fire_info['MaxFRP']

        for lat, lon, frp in zip(lats, lons, frps):
            print(f"Date: {date}, Coordinates: ({lat}, {lon}), FRP: {frp}")

EEException: Empty date ranges not supported for the current operation.

In [16]:
properties

{'date': '2014-12-30', 'fire_info': {'MaxFRP': [], 'x': [], 'y': []}}

In [1]:
import requests
from datetime import datetime, timedelta 

# Constants
API_ENDPOINT = "YOUR_API_ENDPOINT"  # Replace with the actual MODIS API endpoint
API_KEY = "YOUR_API_KEY"            # Replace with your actual NASA API key

# Coordinates for Alaska example
coordinates = {'latitude': 64.2008, 'longitude': -149.4937}

# Dates for analysis (example dates here, adjust as needed)
dates = {
    'pre_fire': (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
    'current': datetime.now().strftime('%Y-%m-%d'),
    'post_fire': (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
}

# MODIS products to fetch
products = {
    'vegetation_indices': "MOD13Q1.006",  # NDVI and EVI
    'burned_area': "MCD64A1.006",
    'land_surface_temp': "MOD11A1.006",
    'thermal_anomalies': "MOD14A1.006"
}

# Function to get MODIS data
def get_modis_data(product_code, latitude, longitude, date, api_key):
    url = f"{API_ENDPOINT}/{product_code}/{latitude},{longitude}/{date}"
    headers = {'Content-Type': 'application/json'}
    params = {'api_key': api_key}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve {product_code} data for {date}: {response.status_code}")
        return None

# Fetch and store data
modis_data = {}
for time_key, date in dates.items():
    modis_data[time_key] = {}
    for feature_name, product_code in products.items():
        modis_data[time_key][feature_name] = get_modis_data(product_code, coordinates['latitude'], coordinates['longitude'], date, API_KEY)

# Check and print the data
for time_key in dates:
    print(f"--- {time_key.upper()} WILDFIRE CONDITIONS ---")
    if all(feature_data is not None for feature_data in modis_data[time_key].values()):
        for feature_name, data in modis_data[time_key].items():
            print(f"{feature_name.replace('_', ' ').title()}:")
            print(data)  # Replace this print with actual data processing
    else:
        print(f"Data retrieval for {time_key} was incomplete.")

MissingSchema: Invalid URL 'YOUR_API_ENDPOINT/MOD13Q1.006/64.2008,-149.4937/2023-10-15': No scheme supplied. Perhaps you meant https://YOUR_API_ENDPOINT/MOD13Q1.006/64.2008,-149.4937/2023-10-15?

### START LEARNING: T76

#### START LEARNING: T76 using the matlab processed dataset (already made sequences)

In [1]:
# Function to prepare sequences
def prepare_sequences(loc, sequence_length, target, features, season, res):
    # Reading the csv file
    if res > 1:
        resolution = 'res'+str(res)+'D_'
    else:
        resolution = ''    

    df = pd.read_csv(f'Air2UG_alltimeseries_{loc}_MERRA2data_{target}_{resolution}SeqLen{str(sequence_length)}_{season}.csv')

    # Initialize an empty list to hold feature columns
    feature_columns = []
    
    if len(features) > 0 and sequence_length > 1:
        # Loop through each feature and sequence length to get the corresponding columns
        for feature in features:
            for i in range(sequence_length):
                column_name = f"{feature}_{i+1}"
                if column_name in df.columns:
                    feature_columns.append(column_name)         
    elif len(features) > 0 and sequence_length == 1:
        # Loop through each feature and sequence length to get the corresponding columns
        for feature in features:
            for i in range(sequence_length):
                column_name = f"{feature}"
                if column_name in df.columns:
                    feature_columns.append(column_name)        
    else:
        feature_columns = []
        
    # Extract time info
    df['Date'] = df['Date'].apply(pd.to_datetime, errors='coerce')
    df['Year'] = df['Date'].dt.year
    feature_columns.append('Year')
    df['Month'] = df['Date'].dt.month
    feature_columns.append('Month')
    df['Day'] = df['Date'].dt.day
    feature_columns.append('Day')
    
    # Extract the feature columns for X
    X = df[feature_columns].values
    
    # Extract the target column for y
    y = df[target].values
    
    return X, y, df

In [2]:
# load data into pandas dataframe
import pandas as pd
import numpy as np
# !pip install sktime
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
import time

# !pip install PrettyTable
from prettytable import PrettyTable

models = [{'model': LinearRegression(), 'params': {}},
          {'model': KNeighborsRegressor(n_jobs=-1), 'params': {'n_neighbors': range(3,23,3),
                                                      'weights': ['uniform', 'distance'],
                                                      'p': [1, 2]}},
          {'model': DecisionTreeRegressor(), 'params': {'max_depth': range(3,23,3),
                                                        'min_samples_split': [2, 4, 6, 8],
                                                        'min_samples_leaf': [1, 2, 3, 5]}},
          {'model': RandomForestRegressor(verbose=0, n_jobs=-1), 'params': {'n_estimators': [10, 20, 50, 70],
                                                        'max_depth': range(3,23,3),
                                                        'min_samples_split': [2, 4, 6],
                                                        'min_samples_leaf': [2, 4, 6]}},
          {'model':LGBMRegressor(n_jobs=-1, verbose=-1,force_col_wise=True), 'params': {'learning_rate': [0.1, 0.01, 0.001],
                                                           'n_estimators': [100,150,200,250,350,500,1000],
                                                           'num_leaves': [20, 31, 40, 60, 80],
                                                           'min_data_in_leaf': range(50,300, 100)}},
          {'model':SVR(verbose=0), 'params':  {'C': [0.1, 1], #10, 100
                                      'gamma': [0.1, 0.01],#1, , 0.001
                                      'kernel': ['linear', 'rbf', 'sigmoid']}},# 'poly',
          {'model':MLPRegressor(verbose=False,max_iter=20000), 'params':  {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,50), (75,35)],
                                               'activation': ['relu','tanh'],#,'logistic'
                                               'alpha': [0.0001, 0.05]}}
          ]

# Function to scale features
def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled


In [19]:
import pandas as pd
loc = 'Toolik'
sequence_length = 1
features = ''
trgt = 'T0'
res = 30
print(f'Res.;Season;sequence length;Target;Dataset size;# years;Training set size;# years')
for season in ['Winter','Spring','Summer','Autumn']:
        X, y, _ = prepare_sequences(loc, sequence_length, trgt, features, season, res)
        # Split into train and test sets
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        print(f'{res};{season};{sequence_length};{trgt};{X.shape[0]};{np.unique(X[:,-3]).shape[0]};{X_train.shape[0]};{np.unique(X_train[:,-3]).shape[0]}')

Res.;Season;sequence length;Target;Dataset size;# years;Training set size;# years
30;Winter;1;T0;1489;17;1191;14
30;Spring;1;T0;1564;17;1251;14
30;Summer;1;T0;1544;17;1235;14
30;Autumn;1;T0;1456;16;1164;13


### direct process

In [72]:
#Time+Tair | TIME+Remote sensing
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
from tsai.models.MINIROCKET import MiniRocketRegressor
from sklearn.metrics import mean_squared_error, make_scorer
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# sequence length
sequence_length = 1
# resolution
loc = 'Toolik'
print(f'Res;seqLen;Season;trgt;model;Feature;bestparam;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2')
# Prepare sequences
for res in [14, 30]:
    for season in ['Winter','Spring','Summer','Autumn']:
        for trgt in ['T0','T16','T31','T46','T76','T97']:  
            for model in models:
                for features in [['Tair'], ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']]:
                    print(f'{res};{sequence_length};{season};{trgt}',end=';')
                    print(str(model['model']).split('(')[0],end=';')
                    print(f'{features}',end=';')
                    X, y, _ = prepare_sequences(loc, sequence_length, trgt, features, season, res)
                #     print(X.shape)

                    # Split into train and test sets
                    train_size = int(len(X) * 0.8)
                    X_train, X_test = X[:train_size], X[train_size:]
                    y_train, y_test = y[:train_size], y[train_size:]

                    # Scale features
                    X_train_scaled, X_test_scaled = scale_features(X_train, X_test)   

                    # hyperparameter tuning
                    if str(model['model']).split('(')[0] != 'LinearRegression' and str(model['model']).split('(')[0] != 'LGBMRegressor':
                        model_final = HalvingGridSearchCV(model['model'], model['params'], cv=10, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)#n_iter=15,
                    elif str(model['model']).split('(')[0] == 'LGBMRegressor':
                        model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
                    else:
                        model_final = model['model']

                    # Train model
                    start_train = time.time()
                    model_final.fit(X_train_scaled, y_train)
                    end_train = time.time()

                    # Evaluate training performance
                    y_pred_train = model_final.predict(X_train_scaled)
                    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
                    r2_train = r2_score(y_train, y_pred_train)
                    mae_train = mean_absolute_error(y_train, y_pred_train)
                    explained_variance_train = explained_variance_score(y_train, y_pred_train)
                    train_duration = end_train - start_train

                    if str(model['model']).split('(')[0] != 'LinearRegression' and str(model['model']).split('(')[0] != 'LGBMRegressor':
                        print(f'{model_final.best_params_};{train_duration};{rmse_train};{mae_train};{explained_variance_train};{r2_train}',end=';')
                    elif str(model['model']).split('(')[0] == 'LGBMRegressor':
                        print(f'learning_rate: 0.01, min_data_in_leaf: 50, n_estimators: 1000, num_leaves: 60;{train_duration};{rmse_train};{mae_train};{explained_variance_train};{r2_train}',end=';')
                    else:
                        print(f'-;{train_duration};{rmse_train};{mae_train};{explained_variance_train};{r2_train}',end=';')

                    # Test model
                    start_test = time.time()
                    y_pred_test = model_final.predict(X_test_scaled)
                    end_test = time.time()

                    # Evaluate testing performance
                    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
                    r2_test = r2_score(y_test, y_pred_test)
                    mae_test = mean_absolute_error(y_test, y_pred_test)
                    explained_variance_test = explained_variance_score(y_test, y_pred_test)
                    test_duration = end_test - start_test

                    print(f'{test_duration};{rmse_test};{mae_test};{explained_variance_test};{r2_test}')

                    # Write results to a CSV file
            #         formatted_dates = np.array(['-'.join(map(str, row)) for row in X_test])
            #         formatted_dates_df = pd.DataFrame(formatted_dates, columns=['Date'])
            #         y_test_df = pd.DataFrame(y_test, columns=['actual'])
            #         y_preds = pd.DataFrame(y_pred, columns=['preds'])
            #         results = pd.concat([formatted_dates_df, y_test_df, y_preds.reset_index(drop=True)], axis=1)
            #         results.to_csv(f'Results2/Preliminary{loc}_{trgt}_res{res}D_features{sequence_length}_LGBMRegressor_{season}_results.csv', index=False)
            print(f'')
        print(f'')
    print(f'')
    print(f'')
        

Res;seqLen;Season;trgt;model;Feature;bestparam;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2
14;1;Winter;T0;LinearRegression;['Tair'];-;0.001079559326171875;5.449657058405619;4.481616547055886;0.12472920634964169;0.12472920634964169;8.893013000488281e-05;6.894459178602438;5.528654002743433;-0.15337889764054258;-0.4148924358198056
14;1;Winter;T0;LinearRegression;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];-;0.0011317729949951172;3.09808620859885;2.3380362551284715;0.7171272665860873;0.7171272665860873;0.00014400482177734375;3.5603105819228307;2.7553006849091686;0.6320682227286527;0.6226895914184647
14;1;Winter;T0;KNeighborsRegressor;['Tair'];{'n_neighbors': 21, 'p': 1, 'weights': 'uniform'};0.6144866943359375;3.6396857316483713;2.977012181005138;0.6096095645857088;0.6095801749621248;0.012340545654296875;8.292731873123127;6.500847152759883;-0.7065516584746745;-1.0470019816472806
14;1;Winter;T0;KNeighborsRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'n_neighbors'

14;1;Winter;T16;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50)};17.625663995742798;0.26228949654088574;0.19383740239901906;0.9969950209484496;0.9969395507217471;0.0004074573516845703;11.166200191398199;8.899215153889362;-1.888706099289557;-6.416550404951042

14;1;Winter;T31;LinearRegression;['Tair'];-;0.0010614395141601562;2.7284185878231884;2.2450299160984435;0.41139427172495235;0.41139427172495246;8.726119995117188e-05;3.355097756210262;2.6646647814974718;0.14373781860445978;-0.17188465705506006
14;1;Winter;T31;LinearRegression;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];-;0.001123666763305664;2.1391608549428494;1.7109052862474778;0.6381826648838851;0.6381826648838851;0.00013756752014160156;3.504648550303262;2.828128217178337;0.4562114354835276;-0.2786846660056703
14;1;Winter;T31;KNeighborsRegressor;['Tair'];{'n_neighbors': 21, 'p': 1, 'weights': 'distance'};0.6003329753875732;0.0;0.0;1

14;1;Winter;T97;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'};0.27515149116516113;1.8544435328424547;1.4283547242923202;0.6794574117631824;0.6781920699827362;0.005590915679931641;1.599844724603069;1.3282799498765623;0.6765427903606908;0.6526709119996412
14;1;Winter;T97;MLPRegressor;['Tair'];{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (75, 35)};21.08669137954712;1.2980482418178085;1.0222625383305333;0.8433897874196635;0.8423292327925456;0.0003573894500732422;2.075348738702597;1.6738588631930276;0.7343405631989754;0.4155226851764673
14;1;Winter;T97;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50)};15.263110637664795;0.19417917936698056;0.14426437662381042;0.9966514465638948;0.9964716241813482;0.0009038448333740234;2.2388938229221;1.8350168400695952;0.32316160161172935;0.31977516326499167


14;1;Spring;T0;LinearRegression;['Tair'

14;1;Spring;T16;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'C': 1, 'gamma': 0.1, 'kernel': 'linear'};0.29867005348205566;2.252104134360049;1.5980771039395039;0.9055560753782684;0.9049595083315651;0.005702495574951172;1.886042885101589;1.5232612911156882;0.9414978098416565;0.936319577358247
14;1;Spring;T16;MLPRegressor;['Tair'];{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50, 50)};47.37895226478577;0.34313173617606674;0.2451315421235223;0.9978060536612534;0.9977937561400787;0.0004706382751464844;4.961221268584863;3.9962415455836005;0.6109221049614009;0.5593639720917191
14;1;Spring;T16;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (75, 35)};18.9552960395813;0.4406045462398767;0.30849832893678164;0.9963919657691552;0.9963622764922292;0.00042724609375;3.0895863477412946;2.4482471111944437;0.8659521381968562;0.8291151802106882

14;1;Spring;T31;LinearRegression;['Tair'];-;0.00142025

14;1;Spring;T46;SVR;['Tair'];{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'};0.2990431785583496;1.7405250256006062;1.2976640657260259;0.7909582782062672;0.7909260456285051;0.01253509521484375;2.0494096962663186;1.766382286289692;0.6838127431212333;0.6823863962771848
14;1;Spring;T46;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'};0.3237190246582031;1.4658855620054094;1.1173994506631646;0.8522312572832694;0.8517005555557762;0.013000249862670898;1.228908236341538;1.0461460822839113;0.8910765919447841;0.8857962322728137
14;1;Spring;T46;MLPRegressor;['Tair'];{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50)};36.98278212547302;0.21978968451793185;0.149092108930676;0.9968990393408382;0.9966660890102097;0.0008733272552490234;5.112334581571706;4.514949250026829;-0.9055624190354543;-0.9764237315751529
14;1;Spring;T46;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_s

...

30;1;Spring;T46;LinearRegression;['Tair'];-;0.0019145011901855469;1.5409959343397364;1.1954536084765484;0.8178069875850037;0.8178069875850037;9.012222290039062e-05;1.3629545243852281;1.0902054363762583;0.8443313629185993;0.8438115516099189
30;1;Spring;T46;LinearRegression;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];-;0.0013427734375;1.2841433097859045;0.9796602150052703;0.8734810095622327;0.8734810095622327;0.00014328956604003906;1.3207858959805603;1.0243520956315115;0.8584667603276843;0.8533267130610622
30;1;Spring;T46;KNeighborsRegressor;['Tair'];{'n_neighbors': 15, 'p': 1, 'weights': 'distance'};0.6658194065093994;0.0;0.0;1.0;1.0;0.015887737274169922;2.022610544812209;1.8049447609415121;0.7402146237444611;0.656037803641857
30;1;Spring;T46;KNeighborsRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'n_neighbors': 9, 'p': 1, 'weights': 'uniform'};0.7147994041442871;0.36988404892098653;0.21826649606566592;0.9895055947132954;0.9895031271458824;0.017428874969482422;1.5

30;1;Summer;T0;MLPRegressor;['Tair'];{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50)};17.493285417556763;0.17734014551436222;0.13803891465606158;0.9962238401503731;0.9962234267336411;0.0004937648773193359;0.5376593528304645;0.4077509294491648;0.9817174972255773;0.9673350837104775
30;1;Summer;T0;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50)};13.54565954208374;0.1504830393485596;0.11430187473322374;0.9972816664547728;0.9972806888592373;0.000743865966796875;1.0205961854576053;0.8183582468057692;0.8897070468033073;0.8823003906946177

30;1;Summer;T16;LinearRegression;['Tair'];-;0.001397848129272461;2.3539833652608118;1.8059274407826904;0.6122121699514234;0.6122121699514234;8.940696716308594e-05;1.3983803607705438;1.3485956535385832;0.9846848734711396;0.7810119601351453
30;1;Summer;T16;LinearRegression;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];-;0.001038551330566406

30;1;Summer;T76;SVR;['Tair'];{'C': 1, 'gamma': 0.1, 'kernel': 'linear'};0.3271312713623047;0.3223864043704163;0.22348693179541473;0.8587405856746099;0.8584022503591191;0.003909111022949219;0.307512320408765;0.2470796318350145;0.7019681599339964;0.6720776173079752
30;1;Summer;T76;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'C': 1, 'gamma': 0.1, 'kernel': 'linear'};0.505744457244873;0.2432393992272953;0.18891479965940058;0.9195460467597858;0.9193933711313086;0.0043354034423828125;0.33638863842295014;0.2547216135511239;0.6774402326353772;0.607600307655527
30;1;Summer;T76;MLPRegressor;['Tair'];{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (100, 50)};4.249550819396973;0.08484136214707295;0.05380635259645322;0.9902549232003904;0.9901934063632343;0.0005440711975097656;0.7290353897174326;0.5597241305950538;0.028839027527694072;-0.8430759916661026
30;1;Summer;T76;MLPRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'activation': 'relu', 'alpha': 0.0001, 'hi

30;1;Autumn;T31;LGBMRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];learning_rate: 0.01, min_data_in_leaf: 50, n_estimators: 1000, num_leaves: 60;1.1409728527069092;0.19982106074604294;0.12007927414003608;0.9896987363205494;0.9896987363205494;0.0014941692352294922;1.7194932865615673;1.2072289151832007;0.7145868994548179;0.6638984943802666
30;1;Autumn;T31;SVR;['Tair'];{'C': 1, 'gamma': 0.1, 'kernel': 'linear'};0.25122785568237305;1.191818092846046;0.7992798274630379;0.6357634217137416;0.6335388569362066;0.0043184757232666016;2.768473158949959;2.223514889320275;0.5561771621785507;0.12873529029998942
30;1;Autumn;T31;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];{'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'};0.2554152011871338;1.1765586910656407;0.759550109379476;0.6467586146337603;0.6428627285989268;0.004778623580932617;2.97015748466862;2.4430428908717317;0.5388634584511127;-0.002832599595277596
30;1;Autumn;T31;MLPRegressor;['Tair'];{'activation': 'tanh', 'alpha': 0.05

30;1;Autumn;T76;LGBMRegressor;['Tair'];learning_rate: 0.01, min_data_in_leaf: 50, n_estimators: 1000, num_leaves: 60;1.671391487121582;0.12534148567898523;0.06472705673995652;0.8704248929502629;0.8704248929502629;0.001756429672241211;0.20880804163731234;0.1337378787807871;-0.8511563351903706;-1.0651114004823143
30;1;Autumn;T76;LGBMRegressor;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];learning_rate: 0.01, min_data_in_leaf: 50, n_estimators: 1000, num_leaves: 60;2.051130771636963;0.09634300521026752;0.03958718602310326;0.9234452260921402;0.9234452260921402;0.0019266605377197266;0.1354987858553264;0.08178450413404238;0.13175378595906673;0.13039935911200096
30;1;Autumn;T76;SVR;['Tair'];{'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'};0.1726081371307373;0.28457109029726685;0.1407496843600095;0.3414169961222201;0.3320969053332835;0.0020318031311035156;0.13687551092604355;0.11124087470534724;0.4851321335910306;0.11263856471176237
30;1;Autumn;T76;SVR;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLM

## Impact of input length on performance

In [3]:
# def BestModelInput_switch(trgt, season):

#     if trgt == 'T0':
#         if season == 'Winter':
#             feature = ['Tair']
#             model_final = LinearRegression()
#         if season == 'Spring':
#             feature = ['Tair']
#             model_final = LinearRegression()
#         if season == 'Summer':
#             feature = ['Tair']
#             model_final = RandomForestRegressor(max_depth = 15, min_samples_leaf = 4, min_samples_split = 4, n_estimators = 20, verbose=0, n_jobs=-1)
#         if season == 'Autumn':
#             feature = ['Tair']
#             model_final = LinearRegression()

#     elif trgt == "T16":
#         if season == 'Winter':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Spring':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = SVR(C = 1, gamma = 0.1, kernel = 'linear', verbose=0)
#         if season == 'Summer':
#             feature = ['Tair']
#             model_final = RandomForestRegressor(max_depth = 9, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 70, verbose=0, n_jobs=-1)
#         if season == 'Autumn':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)    

#     elif trgt == "T31":
#         if season == 'Winter':
#             feature = ['Tair']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Spring':
#             feature = ['Tair']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Summer':
#             feature = ['Tair']
#             model_final = RandomForestRegressor(max_depth = 12, min_samples_leaf = 2, min_samples_split = 4, n_estimators = 20, verbose=0, n_jobs=-1)
#         if season == 'Autumn':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)    

#     elif trgt == "T46":
#         if season == 'Winter':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = SVR(C = 0.1, gamma = 0.1, kernel = 'rbf', verbose=0)
#         if season == 'Spring':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Summer':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = MLPRegressor(activation = 'relu', alpha = 0.0001, hidden_layer_sizes = (75, 35), verbose=False, max_iter=20000)
#         if season == 'Autumn':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = RandomForestRegressor(max_depth = 18, min_samples_leaf = 2, min_samples_split = 6, n_estimators = 20, verbose=0, n_jobs=-1)   

#     elif trgt == "T76":
#         if season == 'Winter':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = RandomForestRegressor(max_depth = 9, min_samples_leaf = 4, min_samples_split = 6, n_estimators = 70, verbose=0, n_jobs=-1) 
#         if season == 'Spring':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Summer':
#             feature = ['Tair']
#             model_final = SVR(C = 1, gamma = 0.1, kernel = 'linear', verbose=0)
#         if season == 'Autumn':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = KNeighborsRegressor(n_neighbors = 18, p = 1, weights = 'distance', n_jobs=-1)   

#     elif trgt == "T97":
#         if season == 'Winter':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = RandomForestRegressor(max_depth = 21, min_samples_leaf = 4, min_samples_split = 2, n_estimators = 70, verbose=0, n_jobs=-1) 
#         if season == 'Spring':
#             feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
#             model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)
#         if season == 'Summer':
#             feature = ['Tair']
#             model_final = SVR(C = 1, gamma = 0.1, kernel = 'linear', verbose=0)
#         if season == 'Autumn':
#             feature = ['Tair']
#             model_final = DecisionTreeRegressor(max_depth = 3, min_samples_leaf = 5, min_samples_split = 6)   
            
#     return feature, model_final

def BestModelInput_switch(trgt, season):
    feature = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
    if trgt == 'T0':
        if season == 'Winter':
            model_final = SVR(C= 0.1, gamma = 0.1, kernel = 'linear')
        if season == 'Spring':
            model_final = SVR(C= 0.1, gamma = 0.1, kernel = 'linear')
        if season == 'Summer':
            model_final = RandomForestRegressor(max_depth = 6, min_samples_leaf = 2, min_samples_split = 6, n_estimators = 50, verbose=0, n_jobs=-1)
        if season == 'Autumn':
            model_final = LinearRegression()

    elif trgt == "T16":
        if season == 'Winter':
            model_final = DecisionTreeRegressor(max_depth = 6, min_samples_leaf = 2, min_samples_split = 8)
        if season == 'Spring':
            model_final = LinearRegression()
        if season == 'Summer':
            model_final = RandomForestRegressor(max_depth = 15, min_samples_leaf = 2, min_samples_split = 6, n_estimators = 70, verbose=0, n_jobs=-1)
        if season == 'Autumn':
            model_final = SVR(C= 0.1, gamma = 0.1, kernel = 'rbf')    

    elif trgt == "T31":
        if season == 'Winter':
            model_final = RandomForestRegressor(max_depth = 15, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 50, verbose=0, n_jobs=-1)
        if season == 'Spring':
            model_final = RandomForestRegressor(max_depth = 6, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 70, verbose=0, n_jobs=-1)
        if season == 'Summer':
            model_final = RandomForestRegressor(max_depth = 9, min_samples_leaf = 2, min_samples_split = 4, n_estimators = 70, verbose=0, n_jobs=-1)
        if season == 'Autumn':
            model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)    

    elif trgt == "T46":
        if season == 'Winter':
            model_final = SVR(C = 1, gamma = 0.1, kernel = 'rbf', verbose=0)
        if season == 'Spring':
            model_final = LinearRegression()
        if season == 'Summer':
            model_final = MLPRegressor(activation = 'relu', alpha = 0.0001, hidden_layer_sizes = (75, 35), verbose=False, max_iter=20000)
        if season == 'Autumn':
            model_final = LinearRegression()   

    elif trgt == "T76":
        if season == 'Winter':
            model_final = SVR(C = 1, gamma = 0.1, kernel = 'rbf', verbose=0)
        if season == 'Spring':
            model_final = DecisionTreeRegressor(max_depth = 3, min_samples_leaf = 3, min_samples_split = 4)
        if season == 'Summer':
            model_final = LinearRegression()
        if season == 'Autumn':
            model_final = KNeighborsRegressor(n_neighbors = 18, p = 1, weights = 'distance', n_jobs=-1)   

    elif trgt == "T97":
        if season == 'Winter':
            model_final = DecisionTreeRegressor(max_depth = 3, min_samples_leaf = 3, min_samples_split = 4) 
        if season == 'Spring':
            model_final = LinearRegression()
        if season == 'Summer':
            model_final = LinearRegression()
        if season == 'Autumn':
            model_final = LGBMRegressor(learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60, n_jobs=-1, verbose=-1,force_col_wise=True)    
            
    return feature, model_final

In [5]:
#Baseline: features for different lengths...
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

loc = 'Toolik'
# resolution
res = 30
# sequence length
print(f'\n\n\n\n\nres;Target;Len;Season;Feature;model;bestparam;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2')
for trgt in ['T0','T16','T31','T46','T76','T97']:
    for season in ['Winter','Spring','Summer','Autumn']:
        for sequence_length in [1,4,7,10,14,21,28,35]:
            features, model_final = BestModelInput_switch(trgt, season)
            print(f'{res};{trgt};{sequence_length};{season};{features}',end=';')
            print(str(model_final).split('(')[0],end=';')
            # Prepare sequences
            X, y, _ = prepare_sequences(loc, sequence_length, trgt, features, season, res)
        #     print(X.shape)

            # Split into train and test sets
            train_size = int(len(X) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Scale features
            X_train_scaled, X_test_scaled = scale_features(X_train, X_test)   

            # Train model
            start_train = time.time()
            model_final.fit(X_train_scaled, y_train)
            end_train = time.time()

            # Evaluate training performance
            y_pred_train = model_final.predict(X_train_scaled)
            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
            r2_train = r2_score(y_train, y_pred_train)
            mae_train = mean_absolute_error(y_train, y_pred_train)
            explained_variance_train = explained_variance_score(y_train, y_pred_train)
            train_duration = end_train - start_train
            hyperparams = str(model_final).split('(')[1].replace('\n', '').replace('              ',' ').replace('force_col_wise=True, ','').replace(', verbose=-1','').replace(')','').replace(', verbose=0','').replace(', n_jobs=-1','')
            print(f'{hyperparams};{train_duration};{rmse_train};{mae_train};{explained_variance_train};{r2_train}',end=';')
            
            # Test model
            start_test = time.time()
            y_pred_test = model_final.predict(X_test_scaled)
            end_test = time.time()

            # Evaluate testing performance
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            r2_test = r2_score(y_test, y_pred_test)
            mae_test = mean_absolute_error(y_test, y_pred_test)
            explained_variance_test = explained_variance_score(y_test, y_pred_test)
            test_duration = end_test - start_test

            print(f'{test_duration};{rmse_test};{mae_test};{explained_variance_test};{r2_test}')

            # Write results to a CSV file
            formatted_dates = np.array(['-'.join(map(str, row)) for row in X_test[:,-3:]])
            formatted_dates_df = pd.DataFrame(formatted_dates, columns=['Date'])
            y_test_df = pd.DataFrame(y_test, columns=['actual'])
            y_preds = pd.DataFrame(y_pred_test, columns=['preds'])
            results = pd.concat([formatted_dates_df, y_test_df, y_preds.reset_index(drop=True)], axis=1)
            if res == 1:
                RES = ''
            else:
                RES = '_res'+ str(res) + 'D'
            model_name = str(model_final).split('(')[0]
            results.to_csv(f'Results3/InputLength_{RES}_{trgt}_{sequence_length}_{season}_{features}_{model_name}_results.csv', index=False)
        print(f'')
    # #PLOT
    # # Assuming model_final is your trained LightGBM model
    # get_lgbm_varimp(model_final, sequence_length, features)






res;Target;Len;Season;Feature;model;bestparam;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2
30;T0;1;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=0.1, gamma=0.1, kernel='linear';0.03819012641906738;2.7358282988970872;2.0895037365447915;0.685722567102471;0.6847521012807081;0.005347728729248047;3.049282783879168;2.513455927515823;0.3075239928763345;0.19978878899993358
30;T0;4;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=0.1, gamma=0.1, kernel='linear';0.047545671463012695;2.7433697144516525;2.0804432412461775;0.6858955518248482;0.6838076205745471;0.007456541061401367;3.161539880132861;2.5681115022530623;0.2558005521925014;0.13978589997505042
30;T0;7;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=0.1, gamma=0.1, kernel='linear';0.0616912841796875;2.7434948933037293;2.068846270629737;0.6873486676516258;0.6845666017979595;0.009861946105957031;3.2291804418938144;2.6185787644112617;0.22921572690016734;0.1004957735705545
30;T0;10;Wi

30;T0;7;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.003023862838745117;2.2170994438615885;1.6702888434856151;0.8997446362697106;0.8997446362697106;0.0003228187561035156;2.6256060182378747;2.096232788762786;0.8498504740215089;0.846557767665528
30;T0;10;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.055138587951660156;2.1977713531981697;1.6481134795929042;0.9011576666132887;0.9011576666132887;0.00017523765563964844;2.69456999904456;2.141946005423324;0.8435432410772982;0.8383913025126769
30;T0;14;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.005940675735473633;2.176132723684257;1.6288308111732759;0.9028690868289508;0.9028690868289508;0.0003249645233154297;2.7291830247449043;2.1733044857038637;0.8410790635414956;0.8338530902307526
30;T0;21;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.012243270874023438;2.1276639267980104;1.599997045238266;0.9072188289061418;0.90

30;T16;21;Summer;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=15, min_samples_leaf=2, min_samples_split=6,         n_estimators=70;0.24283385276794434;0.279045569382814;0.1652483603620364;0.9943613322606807;0.9943613287249893;0.014620780944824219;1.0081662518229377;0.804806667540611;0.9058211128363468;0.8857895275446923
30;T16;28;Summer;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=15, min_samples_leaf=2, min_samples_split=6,         n_estimators=70;0.27495718002319336;0.2517469549065542;0.1561710599836306;0.9954270728940829;0.995426910767925;0.014045238494873047;1.0369413650542456;0.832784579656249;0.9052426864993985;0.8791476409693367
30;T16;35;Summer;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=15, min_samples_leaf=2, min_samples_split=6,         n_estimators=70;0.36043691635131836;0.2531579121958917;0.1563943309280272;0.9953984569335247;0.9953953412363535;0.0174660682678

30;T31;28;Spring;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=6, min_samples_leaf=2, n_estimators=70,         n_jobs=-1;0.24640631675720215;0.5280746432457408;0.3773832212334012;0.9865966038823925;0.9865947334874914;0.013753175735473633;1.7613495104541472;1.4782342191148272;0.920173677182831;0.9107497600817367
30;T31;35;Spring;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=6, min_samples_leaf=2, n_estimators=70,         n_jobs=-1;0.25785279273986816;0.49928578450357164;0.36636177667478076;0.9880275501997287;0.988027446772096;0.016303300857543945;1.7683525884751297;1.4856947391457112;0.9184481969663495;0.9104940038869057

30;T31;1;Summer;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;max_depth=9, min_samples_leaf=2, min_samples_split=4,         n_estimators=70;0.10509800910949707;0.20551516456164654;0.1349331447341253;0.9969225641429126;0.996921133721508;0.01663994789123535;1.026584788709

30;T46;35;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=1, gamma=0.1;0.17273783683776855;1.9685946180434235;1.1258786968345156;0.7268187555281802;0.6911737273974947;0.0532374382019043;2.962606057913954;2.412192927655062;0.01151609892878569;-0.13071905877451728

30;T46;1;Spring;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.0016372203826904297;1.2841433097859045;0.9796602150052703;0.8734810095622327;0.8734810095622327;0.00013256072998046875;1.3207858959805603;1.0243520956315115;0.8584667603276843;0.8533267130610622
30;T46;4;Spring;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.0020110607147216797;1.183797339932057;0.898271799575092;0.8927075185346836;0.8927075185346836;0.00028014183044433594;1.4272428136444744;1.061057863472211;0.840136589391617;0.8287297588624492
30;T46;7;Spring;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;;0.003046751022338867;1.121311792189182;0.8547981125132506;0.903849182644281

30;T76;14;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=1, gamma=0.1;0.10237622261047363;1.2144679342474558;0.5298081956450889;0.8840829208624742;0.8750420447481573;0.01914811134338379;2.4484865247997454;1.926618319243054;0.30012008876226404;0.2408581927325104
30;T76;21;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=1, gamma=0.1;0.12916278839111328;1.471524568581486;0.7233652398434016;0.8361584503847876;0.8164512004676676;0.026154041290283203;2.8232129310504086;2.2990602376173257;0.11326188978835838;-0.00525089800403955
30;T76;28;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=1, gamma=0.1;0.16835999488830566;1.6653169925816282;0.9018074697875796;0.7958663715638308;0.7652055153854067;0.052144527435302734;2.9800580830588226;2.45857467861889;0.036731110755198526;-0.11810965401937268
30;T76;35;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;C=1, gamma=0.1;0.18076133728027344;1.8223557009631963;1.0597662035382984;0.7577081218

30;T97;1;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];DecisionTreeRegressor;max_depth=3, min_samples_leaf=3, min_samples_split=4;0.0022134780883789062;1.5447899790368806;1.2304646796295686;0.7666356330363608;0.7666356330363608;0.00010156631469726562;1.5967154950875952;1.225345055456635;0.628597522011713;0.6252770446220133
30;T97;4;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];DecisionTreeRegressor;max_depth=3, min_samples_leaf=3, min_samples_split=4;0.0057964324951171875;1.540189885611991;1.2302546849161118;0.768089819719551;0.768089819719551;0.0002484321594238281;1.6815515020288176;1.2928120914021433;0.591614212249928;0.5843999745527217
30;T97;7;Winter;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];DecisionTreeRegressor;max_depth=3, min_samples_leaf=3, min_samples_split=4;0.01016378402709961;1.5333421993793797;1.2309150324723672;0.7699750997080557;0.7699750997080557;0.0004489421844482422;1.7374601013195232;1.3580593821335813;0.5986428340792692;0.5574606640

30;T97;10;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60;1.2611632347106934;0.06274045605364337;0.020987524003865537;0.9082970891645543;0.9082970891645543;0.002602815628051758;0.06606443427747102;0.036839136334788226;-2.2531597811843986;-2.327584169143176
30;T97;14;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60;1.5403263568878174;0.06218781425271956;0.019739315097994793;0.910116574683491;0.910116574683491;0.0019876956939697266;0.06801266242876759;0.03422739351884382;-2.4690288482792164;-2.5149724236412663
30;T97;21;Autumn;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;learning_rate=0.01, min_data_in_leaf=50, n_estimators=1000, num_leaves=60;1.5255920886993408;0.05300095227687029;0.015821161299411034;0.935005394930257;0.9350053949302569;0.0030155181884765625;0.05036053933589

### Impact of train size on performance

In [7]:
def get_training_indices(df, num_years):
    """Get indices for the last `num_years` years."""
    max_year = df['Year'].max()
    start_year = max_year - num_years + 1
    return df[df['Year'].between(start_year, max_year)].index    

In [8]:
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# sequence length
sequence_length = 1
# resolution
res = 30
loc = 'Toolik'
print(f'Res;seqLen;season;target;trSize;feature;model;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2')
for season in ['Winter','Spring','Summer','Autumn']:
        for trgt in ['T0','T16','T31','T46','T76','T97']:

            features, model_final = BestModelInput_switch(trgt, season)

            # Get all data for the season
            X_all, y_all, df_all = prepare_sequences(loc, sequence_length, trgt, features, season, res)
            train_size = int(len(X_all) * 0.8)

            # Slice down to only the training set
            X_all_train = X_all[:train_size]
            y_all_train = y_all[:train_size]
            df_all_train = df_all.iloc[:train_size]

            unique_years = df_all_train['Year'].nunique()

            for tr_size in range(1, unique_years + 1):  # Use 1 year, 2 years, ... up to all years for training
                # Get all data for the season - do this everytime to standardize the test set similarly to the training set
                X_all, y_all, df_all = prepare_sequences(loc, sequence_length, trgt, features, season, res)
                train_size = int(len(X_all) * 0.8)

                # Slice down to only the training set
                X_all_train = X_all[:train_size]
                y_all_train = y_all[:train_size]
                df_all_train = df_all.iloc[:train_size]

                X_test = X_all[train_size:]
                y_test = y_all[train_size:]

                unique_years = df_all_train['Year'].nunique()

                print(f'{res};{sequence_length};{season};{trgt};{tr_size}years;{features}',end=';')
                print(str(model_final).split('(')[0],end=';')
                
                indices = get_training_indices(df_all_train, tr_size)
                X_train = X_all_train[indices]
                y_train = y_all_train[indices]

                # Scale features
                X_train_scaled, X_test_scaled = scale_features(X_train, X_test)   

                # Train model
                start_train = time.time()
                model_final.fit(X_train_scaled, y_train)
                end_train = time.time()

                # Evaluate training performance
                y_pred_train = model_final.predict(X_train_scaled)
                rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
                r2_train = r2_score(y_train, y_pred_train)
                mae_train = mean_absolute_error(y_train, y_pred_train)
                explained_variance_train = explained_variance_score(y_train, y_pred_train)
                train_duration = end_train - start_train
                print(f'{train_duration};{rmse_train};{mae_train};{explained_variance_train};{r2_train}',end=';')

                # Test model
                start_test = time.time()
                y_pred_test = model_final.predict(X_test_scaled)
                end_test = time.time()

                # Evaluate testing performance
                rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
                r2_test = r2_score(y_test, y_pred_test)
                mae_test = mean_absolute_error(y_test, y_pred_test)
                explained_variance_test = explained_variance_score(y_test, y_pred_test)
                test_duration = end_test - start_test

                print(f'{test_duration};{rmse_test};{mae_test};{explained_variance_test};{r2_test}')
            print(f'')
        print(f'')


Res;seqLen;season;target;trSize;feature;model;trainDur;RMSE;MAE;Var;R2;testDur;RMSE;MAE;Var;R2
30;1;Winter;T0;1years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0005884170532226562;0.19469560415129658;0.15593619446729023;0.98624169171885;0.9861907286811257;0.0002071857452392578;5.301238803946587;4.21783125983912;-0.34572951823391507;-1.4185995322923493
30;1;Winter;T0;2years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0009889602661132812;0.5264269994561808;0.3689332437564934;0.9837173948518233;0.9836135309785717;0.0005004405975341797;7.686822994182455;6.994329325303378;-0.38653081451286075;-4.085140880473887
30;1;Winter;T0;3years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0023195743560791016;0.9304906932120071;0.7505452643172216;0.956246679948187;0.956194870189147;0.0010709762573242188;4.006392356876815;3.4326727085443145;0.07385187036814378;-0.38138961072886746
30;1;Winter;T0;4years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0030057

30;1;Winter;T31;2years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.06961202621459961;0.151713783047312;0.07482854188344622;0.9981463310865056;0.9980888969402582;0.011258125305175781;3.1520920858361086;2.435764387605009;-0.10498497639228255;-0.4477009486208863
30;1;Winter;T31;3years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.0718531608581543;0.23864434741906904;0.10092174576279284;0.9947315233761336;0.9947124846919136;0.010698318481445312;3.023897224794168;2.522183450628512;-0.2612243245871013;-0.33234015969411
30;1;Winter;T31;4years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.07350659370422363;0.12668837598542523;0.06993579770006667;0.9986125578396382;0.9986124959530343;0.010871410369873047;2.4300497989842387;1.8401404582419931;0.444266254073836;0.13957818437322622
30;1;Winter;T31;5years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.07485055923461914;0.118448023043

30;1;Winter;T76;4years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0037069320678710938;0.6047532862686249;0.3199480738215742;0.9657026193669923;0.9645909793151932;0.0026733875274658203;1.7462908778858144;1.4737967100907137;0.6120374473700033;0.6120363626135976
30;1;Winter;T76;5years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0060389041900634766;0.6006943779648444;0.31116178611862383;0.9641419127042001;0.9634613706711016;0.003366708755493164;1.770006630386877;1.5429882097596292;0.6018139089510559;0.6014272178868205
30;1;Winter;T76;6years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.008816003799438477;0.6250620505076183;0.31239604430195433;0.9721071389945384;0.9717339740000844;0.003949403762817383;1.6944423069485715;1.4797421990227395;0.7035967206091398;0.6347321700784673
30;1;Winter;T76;7years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.011741161346435547;0.6063198831848736;0.3187740143817223;0.9722420174819806;0.9721080577810206;0.004

30;1;Spring;T0;6years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.007456779479980469;1.9753737594461742;1.2054806089153711;0.960860224270839;0.959984222362233;0.0024406909942626953;2.3893625344474017;2.1952681996905596;0.9718902831649675;0.9065244322577386
30;1;Spring;T0;7years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.009437322616577148;1.9430001290538716;1.2630957739366135;0.962452233790763;0.9622978375907738;0.002904176712036133;1.8591930704811157;1.625367771436807;0.9593098883087658;0.9434043572597889
30;1;Spring;T0;8years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.015841245651245117;1.8870996291424749;1.2600010225418001;0.9630918307507778;0.9630252567494377;0.003278970718383789;1.5970282365883068;1.1933307998837341;0.9586467101918519;0.9582401238416046
30;1;Spring;T0;9years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.01835918426513672;1.859240694108745;1.3181867736915684;0.9628994747931067;0.9628710838572028;0.0036776065826416

30;1;Spring;T31;8years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.09902191162109375;0.5235937512508768;0.34281211299476794;0.9890775685869251;0.9890775256883503;0.012884378433227539;1.6007661831504376;1.2745223580377198;0.9321683447892228;0.925451988971588
30;1;Spring;T31;9years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.09872674942016602;0.5294711221339605;0.35402670031414063;0.9883717683750672;0.988368005104624;0.013438224792480469;1.8711179295386573;1.445740971374979;0.909314162670127;0.8981449469597562
30;1;Spring;T31;10years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.10071063041687012;0.5651606419272535;0.3895978101353473;0.9870935239300841;0.9870935198760523;0.012838363647460938;1.7109126530696384;1.3476748664128075;0.9179476924741666;0.9148399446708518
30;1;Spring;T31;11years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.09987306594848633;0.613202588180512

...

30;1;Summer;T31;11years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.10183548927307129;0.16867935544689527;0.09444025492452797;0.9980788642137406;0.9980787254700114;0.013170242309570312;1.0418520755938445;0.8273239361492376;0.8756743167088479;0.8744497771529114
30;1;Summer;T31;12years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.10322690010070801;0.16782306465742983;0.10292401757325882;0.9980264584572293;0.9980256731964284;0.013361930847167969;1.075935182514045;0.8562657088847093;0.8681157089268693;0.8661009230203353
30;1;Summer;T31;13years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.10218334197998047;0.18541967265489953;0.11527786900036883;0.9975369592250051;0.997536766082799;0.013080596923828125;0.9928908360599508;0.7934009200099621;0.8864163219975757;0.88597282363433
30;1;Summer;T31;14years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];RandomForestRegressor;0.10341978073120117;0.206722877

30;1;Summer;T76;12years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0009357929229736328;0.2456073219063933;0.19682939482269055;0.900251431755618;0.900251431755618;0.00010991096496582031;0.36796898423893437;0.27253831819750973;0.655028890140416;0.5304644841340165
30;1;Summer;T76;13years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0009500980377197266;0.2440007860807302;0.1958334400359371;0.9110202393999625;0.9110202393999625;0.00011515617370605469;0.3409898493001792;0.2569451469305768;0.663609747213487;0.5967922025823398
30;1;Summer;T76;14years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0010006427764892578;0.2400199971573782;0.1914433994776593;0.9215129932379325;0.9215129932379325;0.00010967254638671875;0.3240137492341219;0.25143645060222686;0.6696610860348632;0.6359400549727788

30;1;Summer;T97;1years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0006573200225830078;0.02682356069802502

30;1;Autumn;T0;13years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0017511844635009766;2.2412999598347714;1.7051776231450082;0.8981820321707514;0.8981820321707514;0.00011610984802246094;2.5369160251099347;2.0139882512831773;0.8599327918636808;0.8571302260334109

30;1;Autumn;T16;1years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0007562637329101562;2.2657301003209933;1.6555230251555475;0.6417575391550845;0.6295399633442242;0.0008876323699951172;5.4369812864297025;4.215407044550133;0.24148763161416265;-0.11493633610161047
30;1;Autumn;T16;2years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.0013761520385742188;4.654731933257233;3.399713605807731;0.5466102736498132;0.5349186122934333;0.001909494400024414;5.022011321457257;3.9603477811239705;0.15989733976541043;0.048760736601921706
30;1;Autumn;T16;3years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];SVR;0.002748250961303711;4.179237325029981;2.820676283621876;0.6009048816189535;0.599211

30;1;Autumn;T46;4years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0007245540618896484;0.30071896775989543;0.22343127504041832;0.8893212104403281;0.8893212104403281;6.723403930664062e-05;0.8581351516551918;0.7041489316215379;0.30669784513064036;0.025703725866146
30;1;Autumn;T46;5years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0007443428039550781;0.31121976231096465;0.2502548560715115;0.8565762569499811;0.8565762569499811;6.842613220214844e-05;0.7060759996813522;0.5943761791859908;0.382360631102021;0.34039710002972523
30;1;Autumn;T46;6years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0007615089416503906;0.49054630750979183;0.38007185059701687;0.7696945223209971;0.7696945223209971;8.320808410644531e-05;0.7356412926261834;0.587956985753115;0.7026072381639363;0.28400192469438656
30;1;Autumn;T46;7years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LinearRegression;0.0007686614990234375;0.5073065132197446;0.

30;1;Autumn;T97;9years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;1.1906495094299316;0.060317188862223814;0.022142239643182134;0.9011681609059907;0.9011681609059907;0.0016531944274902344;0.0886167544772924;0.045687152451351215;-4.879406187421106;-5.027522503387584
30;1;Autumn;T97;10years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;1.1309804916381836;0.0625067380055147;0.021953553328675417;0.8825256194175397;0.8825256194175397;0.00180816650390625;0.08060305808457947;0.04126964553928346;-3.89459761744957;-3.9866650552094676
30;1;Autumn;T97;11years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;0.9687280654907227;0.05970232659697259;0.021396600349133228;0.8821067758112444;0.8821067758112444;0.0018773078918457031;0.07552188308383753;0.03853246553220724;-3.3753794733479934;-3.377768383240471
30;1;Autumn;T97;12years;['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP'];LGBMRegressor;0.8963479995727539;0.08095816010646854;0.02889508348

In [11]:
#Baseline: TIME only
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# List of features to use
features = ''
# sequence length
sequence_length = 1
# resolution
res = 7
loc = 'Toolik'
print(f'Res:seqLen:Feature:TrainSize:Season:RMSE:R2')
# Prepare sequences
for season in ['Winter','Spring','Summer','Autumn']:
    for tr_size in [1,3/4,1/2,1/4,0]:
        print(f'{res}:{sequence_length}:Time:{tr_size}:{season}',end=':')
        for trgt in ['T0','T16','T31','T46','T76','T97']:
            X, y, _ = prepare_sequences(loc, sequence_length, trgt, features, season, res)
        #     print(X.shape)

            # Split into train and test sets
            train_size = int(len(X) * 0.8)
            if tr_size == 0:
                X_train, X_test = X[:int(np.floor(90))], X[train_size:]
                y_train, y_test = y[:int(np.floor(90))], y[train_size:]                
            else:
                X_train, X_test = X[:int(np.floor(train_size*tr_size))], X[train_size:]
                y_train, y_test = y[:int(np.floor(train_size*tr_size))], y[train_size:]

            # Train final LightGBM model using fused features
            model_final = LGBMRegressor(verbose=-1,force_col_wise=True)
            model_final.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape the data to 2D if needed

            # Make predictions
            y_pred = model_final.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape the data to 2D if needed

            # Evaluate the model
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)
            print(f'{rmse_test}:{r2}',end=':')

        print(f'')


Res:TrainSize:seqLen:Season:Feature:RMSE:R2
7:1:1:Winter:Time:10.019852996004552:-0.6844820053562999:6.6977770280749125:-0.72247352587151:4.462820856770888:-0.6238184731030998:2.11571845660505:0.5086088956048325:1.8051706082349497:0.6306458137880641:1.6766852632295486:0.6290854938233288:
7:0.75:1:Winter:Time:9.814186408440088:-0.6160406517767063:7.266330887114187:-1.0273163612891807:4.853563247731016:-0.9206134080035726:2.2470396271791007:0.44571515602419054:1.7344283762082493:0.659027574772886:1.515013542411485:0.6971666129494432:
7:0.5:1:Winter:Time:10.314572255599117:-0.7850324731073441:6.883111405617229:-0.8191176266857303:3.647207958873599:-0.08452513940192774:3.408776397914864:-0.2755825494218085:3.4945489162020418:-0.3841678773151225:3.274924091914998:-0.415054609319494:
7:0.25:1:Winter:Time:9.888751174425062:-0.6406901632719688:6.647931787744446:-0.696931430412804:4.107488315613764:-0.3755337257817264:2.294540115246412:0.4220332552576017:1.8221978425744112:0.6236451003222434:1.

In [14]:
#Baseline: select features
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# List of features to use
features = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
# sequence length
sequence_length = 1
# resolution
res = 7
loc = 'Toolik'
print(f'Res:seqLen:Feature:TrainSize:Season:RMSE:R2')
# Prepare sequences
for season in ['Winter','Spring','Summer','Autumn']:
    for tr_size in [1,3/4,1/2,1/4,0]:
        print(f'{res}:{sequence_length}:{features}:{tr_size}:{season}',end=':')
        for trgt in ['T0','T16','T31','T46','T76','T97']:
            X, y, _ = prepare_sequences(loc, sequence_length, trgt, features, season, res)
        #     print(X.shape)

            # Split into train and test sets
            train_size = int(len(X) * 0.8)
            if tr_size == 0:
                X_train, X_test = X[:int(np.floor(90))], X[train_size:]
                y_train, y_test = y[:int(np.floor(90))], y[train_size:]                
            else:
                X_train, X_test = X[:int(np.floor(train_size*tr_size))], X[train_size:]
                y_train, y_test = y[:int(np.floor(train_size*tr_size))], y[train_size:]

            # Train final LightGBM model using fused features
            model_final = LGBMRegressor(verbose=-1,force_col_wise=True)
            model_final.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape the data to 2D if needed

            # Make predictions
            y_pred = model_final.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape the data to 2D if needed

            # Evaluate the model
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)
            print(f'{rmse_test}:{r2}',end=':')

            # Write results to a CSV file
    #         formatted_dates = np.array(['-'.join(map(str, row)) for row in X_test])
    #         formatted_dates_df = pd.DataFrame(formatted_dates, columns=['Date'])
    #         y_test_df = pd.DataFrame(y_test, columns=['actual'])
    #         y_preds = pd.DataFrame(y_pred, columns=['preds'])
    #         results = pd.concat([formatted_dates_df, y_test_df, y_preds.reset_index(drop=True)], axis=1)
    #         results.to_csv(f'Results2/Preliminary{loc}_{trgt}_res{res}D_{time}{sequence_length}_LGBMRegressor_{season}_results.csv', index=False)
        print(f'')


Res:seqLen:Feature:TrainSize:Season:RMSE:R2
7:1:['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:1:Winter:3.7198342251353087:0.7678381542046916:3.433747181385108:0.5472819437475749:2.956878108759022:0.2871709115967733:1.9053306268912458:0.6014780580712136:2.304307811187044:0.3981510569188952:2.1062300298967758:0.41469487738677335:
7:1:['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:0.75:Winter:5.81268206954176:0.43311258593464863:5.806778308264995:-0.29467772510915125:4.2671460878776335:-0.48454577201379845:1.9210362212357426:0.5948809663536121:1.6301678016461663:0.6987887835907982:1.4467059598323737:0.7238587634566358:
7:1:['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:0.5:Winter:6.278892695527048:0.33853053883306816:5.507588966228792:-0.16470040591638346:2.6409345911462423:0.4313646434479559:2.6972628067856945:0.20134710812202028:3.1546604371789244:-0.1280069014080194:3.0027836686599225:-0.18964891948319518:
7:1:['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:0.25:Winter:

In [14]:
#Baseline: features for different lengths...
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# List of features to use
features = ['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']
# resolution
res = 7
# sequence length
for sequence_length in [70]:#[1,4,7,10,14,21,28,35,49,56,63,70]
    print(f'\n\n\n\n\nLen:Target:Feature:Season:RMSEtesting')
    # Prepare sequences
    for trgt in ['T0','T16','T31','T46','T76','T97']:
        for season in ['Winter','Spring','Summer','Autumn']:

            X, y, _ = prepare_sequences(sequence_length, trgt, features, season, res)
        #     print(X.shape)

            # Split into train and test sets
            train_size = int(len(X) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Train final LightGBM model using fused features
            model_final = LGBMRegressor(verbose=0,force_col_wise=True)
            model_final.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape the data to 2D if needed

            # Make predictions
            y_pred = model_final.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape the data to 2D if needed

            # Evaluate the model
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
            print(f'{sequence_length}:{trgt}(res={res}D):{features}:{season}: {rmse_test}')

            # Write results to a CSV file
            formatted_dates = np.array(['-'.join(map(str, row)) for row in X_test])
            formatted_dates_df = pd.DataFrame(formatted_dates, columns=['Date'])
            y_test_df = pd.DataFrame(y_test, columns=['actual'])
            y_preds = pd.DataFrame(y_pred, columns=['preds'])
            results = pd.concat([formatted_dates_df, y_test_df, y_preds.reset_index(drop=True)], axis=1)
            if res == 1:
                RES = ''
            else:
                RES = '_res'+ str(res) + 'D'
            results.to_csv(f'Results2/Preliminary_{trgt}{RES}_SNODP{sequence_length}_LGBMRegressor_{season}_results.csv', index=False)
    # #PLOT
    # # Assuming model_final is your trained LightGBM model
    # get_lgbm_varimp(model_final, sequence_length, features)






Len:Target:Feature:Season:RMSEtesting
70:T0(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Winter: 3.7810662351265405
70:T0(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Spring: 2.0453983357076178
70:T0(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Summer: 1.3241806649452
70:T0(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Autumn: 2.3962388597994617
70:T16(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Winter: 3.9520605785677874
70:T16(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Spring: 2.4227915133197953
70:T16(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Summer: 1.4901969207222545
70:T16(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Autumn: 3.2788997428741755
70:T31(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Winter: 2.728084109155547
70:T31(res=14D):['SNODP', 'SWGDN', 'T2M', 'SWLAND', 'TLML', 'SLP']:Spring: 2.6080250119825816
70:T31(res=14D):['SNODP',

### decomposed process many features : proposed technique searching for best features using GA

In [4]:
def modellearning(sequence_length, trgt, selected_features, season, res):
    X, y, _ = prepare_sequences(sequence_length, trgt, selected_features, season, res)
#     print(X.shape)

    # Split into train and test sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train final LightGBM model using fused features
    model_final = LGBMRegressor(verbose=0,force_col_wise=True)
    model_final.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape the data to 2D if needed

    # Make predictions
    y_pred = model_final.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape the data to 2D if needed

    # Evaluate the model
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse_test

In [ ]:
# !pip install deap
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from deap import base, creator, tools, algorithms
import random

# Dictionary to store RMSE for evaluated feature combinations
evaluated_combinations = {}

# Objective function for GA
def evalOneMax(individual, all_feature_names, target, sequence_length, season, res):
    selected_features = [all_feature_names[i] for i, is_selected in enumerate(individual) if is_selected]
    selected_features_tuple = tuple(sorted(selected_features))
    
    # Ensure at least 1 feature are selected
    if len(selected_features) < 0:
        return float('inf'),
    
    # Always proceed with model training, bypassing the cache check
    rmse = modellearning(sequence_length, target, selected_features, season, res)

    return rmse,

def optimize_features(target, sequence_length, season, res, nmbr_population, nmbr_generation, known_good_features=None):
    best_features = []  # Initialize to an empty list
    best_rmse = float('inf')  # Initialize to positive infinity
    
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(all_feature_names))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalOneMax, all_feature_names=all_feature_names, target=target, sequence_length=sequence_length, season=season, res=res)

    population = toolbox.population(n=nmbr_population)

    # Seed the population with a known good individual, if provided
    if known_good_features:
        good_individual = [0] * len(all_feature_names)
        for feature in known_good_features:
            idx = all_feature_names.index(feature)
            good_individual[idx] = 1
        population[0] = creator.Individual(good_individual)
        population[0].fitness.values = evalOneMax(population[0], all_feature_names, target, sequence_length, season, res)

    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=nmbr_generation, verbose=False)

    # Extract the best individual
    best_ind = tools.selBest(population, k=1)[0]
    best_features = [all_feature_names[i] for i, is_selected in enumerate(best_ind) if is_selected]
    best_rmse = best_ind.fitness.values[0]
    
    return best_features, best_rmse

# Your data loading here
res = 14 #change this: 1, 7, 14, 30
sequence_length = 1
all_feature_names = ['Tair', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M', 'SLP']
known_good_features = ['SNODP','Tair']  # Replace with your own known good features
nmbr_population = 500
nmbr_generation = 10

## Run optimization
for season in ['Autumn', 'Spring', 'Summer', 'Winter']:#
    print(f'*** Start optimization: {res}, {season}', end='')
    for target in ['T0', 'T16', 'T31', 'T46', 'T76', 'T97']:
        best_features, best_rmse = optimize_features(target, sequence_length, season, res, nmbr_population, nmbr_generation, known_good_features)
        print(f"\n{res}, {season}, {target}, {best_features}, {best_rmse}")


*** Start optimization: 14, Autumn
14, Autumn, T0, ['T2M', 'GHTSKIN', 'TLML', 'EVPSOIL', 'SLP'], 1.5020732696048098


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



14, Autumn, T16, ['Tair', 'SWGDN', 'SWLAND', 'GHTSKIN', 'TLML', 'EVPSOIL'], 2.666874994938504


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



14, Autumn, T31, ['SWGDN', 'SWLAND', 'GHTSKIN', 'TLML', 'QV2M', 'SLP'], 1.5929082535204033


...

/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



14, Spring, T0, ['SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 'TLML', 'EVPSOIL', 'SLP'], 1.67961768484642


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



14, Spring, T16, ['LWGAB', 'T2M', 'SWLAND', 'TLML', 'LWLAND', 'QV2M', 'SLP'], 2.0510703472759326


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [4]:
# !pip install deap
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from deap import base, creator, tools, algorithms
import random

# Dictionary to store RMSE for evaluated feature combinations
evaluated_combinations = {}

# Objective function for GA
def evalOneMax(individual, all_feature_names, target, sequence_length, season, res):
    selected_features = [all_feature_names[i] for i, is_selected in enumerate(individual) if is_selected]
    selected_features_tuple = tuple(sorted(selected_features))
    
    # Ensure at least 1 feature are selected
    if len(selected_features) < 0:
        return float('inf'),
    
    # Always proceed with model training, bypassing the cache check
    rmse = modellearning(sequence_length, target, selected_features, season, res)

    return rmse,

def optimize_features(target, sequence_length, season, res, nmbr_population, nmbr_generation, known_good_features=None):
    best_features = []  # Initialize to an empty list
    best_rmse = float('inf')  # Initialize to positive infinity
    
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(all_feature_names))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalOneMax, all_feature_names=all_feature_names, target=target, sequence_length=sequence_length, season=season, res=res)

    population = toolbox.population(n=nmbr_population)

    # Seed the population with a known good individual, if provided
    if known_good_features:
        good_individual = [0] * len(all_feature_names)
        for feature in known_good_features:
            idx = all_feature_names.index(feature)
            good_individual[idx] = 1
        population[0] = creator.Individual(good_individual)
        population[0].fitness.values = evalOneMax(population[0], all_feature_names, target, sequence_length, season, res)

    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=nmbr_generation, verbose=False)

    # Extract the best individual
    best_ind = tools.selBest(population, k=1)[0]
    best_features = [all_feature_names[i] for i, is_selected in enumerate(best_ind) if is_selected]
    best_rmse = best_ind.fitness.values[0]
    
    return best_features, best_rmse

# Your data loading here
res = 7 #change this: 1, 7, 14, 30
sequence_length = 1
all_feature_names = ['Tair', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M', 'SLP']
known_good_features = ['SNODP','Tair']  # Replace with your own known good features
nmbr_population = 500
nmbr_generation = 10

## Run optimization
for season in ['Summer']:#['Autumn', 'Spring', 'Summer']:#, 'Winter'
    print(f'*** Start optimization: {res}, {season}', end='')
    for target in ['T0', 'T16', 'T31', 'T46', 'T76', 'T97']:
        best_features, best_rmse = optimize_features(target, sequence_length, season, res, nmbr_population, nmbr_generation, known_good_features)
        print(f"\n{res}, {season}, {target}, {best_features}, {best_rmse}")


*** Start optimization: 7, Summer

/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



7, Summer, T0, ['Tair', 'LWGAB', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'TS', 'QV2M'], 0.6777126076769224


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



7, Summer, T16, ['SNODP', 'SWGDN', 'LWGAB', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TS', 'QV2M'], 0.868811191537385


...


7, Summer, T46, ['GHTSKIN', 'LWLAND', 'QV2M'], 1.3579162669544054


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



7, Summer, T76, ['Tair', 'SNODP', 'SWGDN', 'GHTSKIN', 'HFLUX', 'EVPSOIL', 'LWLAND', 'SLP'], 0.33132696426984953


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



7, Summer, T97, ['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'LWLAND'], 0.4177147049853185


### Results using optimized features with different lengths

In [6]:
def bestfeatures_Weekly(month, trgt):
    if month == 'Autumn':
        if trgt == 'T0':
            features = ['Tair', 'SNODP', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'EVPSOIL']
            return features
        elif trgt == 'T16':
            features = ['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'TSH', 'SLP']
            return features
        elif trgt == 'T31':
            features = ['T2M', 'SWLAND', 'GHTSKIN', 'TLML', 'QV2M', 'SLP']
            return features
        elif trgt == 'T46':
            features = ['Tair', 'T2M', 'SWLAND', 'GHTSKIN', 'TLML', 'QV2M', 'SLP']
            return features
        elif trgt == 'T76':
            features = ['Tair', 'SWGDN', 'T2M', 'SWLAND', 'GHTSKIN', 'TS']
            return features
        elif trgt == 'T97':
            features = ['Tair', 'SNODP', 'LWGAB', 'GHTSKIN', 'TSH', 'LWLAND', 'SLP']
            return features

    if month == 'Spring':
        if trgt == 'T0':
            features = ['Tair', 'SNODP', 'LWGAB', 'T2M', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M']
            return features
        elif trgt == 'T16':
            features = ['Tair', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'LWLAND', 'QV2M', 'SLP']
            return features
        elif trgt == 'T31':
            features = ['T2M', 'SPEED', 'TLML', 'EVPSOIL', 'SLP']
            return features
        elif trgt == 'T46':
            features = ['Tair', 'SNODP', 'LWGAB', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'EVPSOIL', 'LWLAND', 'TS', 'SLP']
            return features
        elif trgt == 'T76':
            features = ['Tair', 'SNODP', 'SWGDN', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'TS', 'SLP']
            return features
        elif trgt == 'T97':
            features = ['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'SLP']
            return features   


    if month == 'Summer':
        if trgt == 'T0':
            features = ['Tair', 'LWGAB', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'TS', 'QV2M']
            return features
        elif trgt == 'T16':
            features = ['SNODP', 'SWGDN', 'LWGAB', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TS', 'QV2M']
            return features
        elif trgt == 'T31':
            features = ['Tair', 'LWGAB', 'SWLAND', 'GHTSKIN', 'HFLUX', 'SPEED', 'TSH', 'TS']
            return features
        elif trgt == 'T46':
            features = ['GHTSKIN', 'LWLAND', 'QV2M']
            return features
        elif trgt == 'T76':
            features = ['Tair', 'SNODP', 'SWGDN', 'GHTSKIN', 'HFLUX', 'EVPSOIL', 'LWLAND', 'SLP']
            return features
        elif trgt == 'T97':
            features = ['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'LWLAND']
            return features   
    
    if month == 'Winter':
        if trgt == 'T0':
            features = ['SWGDN', 'SWLAND', 'TLML', 'EVPSOIL', 'TS', 'QV2M']
            return features
        elif trgt == 'T16':
            features = ['SWGDN', 'SWLAND', 'TSH', 'EVPSOIL', 'SLP']
            return features
        elif trgt == 'T31':
            features = ['Tair', 'SWGDN', 'LWGAB', 'SPEED', 'LWLAND', 'SLP']
            return features
        elif trgt == 'T46':
            features = ['SNODP', 'LWGAB', 'T2M', 'SWLAND', 'TLML', 'SLP']
            return features
        elif trgt == 'T76':
            features = ['SLP']
            return features
        elif trgt == 'T97':
            features = ['Tair', 'HFLUX', 'SPEED', 'LWLAND', 'SLP']
            return features
    


In [8]:
#Baseline: TIME+Tair only
# Importing required libraries
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# List of features to use
# sequence length
sequence_length = 1
# resolution
res = 7
season = 'Autumn'

print(f'resolution:Season:Target:Feature:RMSEtesting')
for sequence_length in [1,4,7,10,14,21,28,35,42]:#,49,56,63,70
    for trgt in ['T0','T16','T31','T46','T76','T97']:
    # Prepare sequences
        features = bestfeatures_Weekly(season, trgt)
        X, y, _ = prepare_sequences(sequence_length, trgt, features, season, res)
    #     print(X.shape)

        # Split into train and test sets
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # Train final LightGBM model using fused features
        model_final = LGBMRegressor(verbose=0,force_col_wise=True)
        model_final.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Reshape the data to 2D if needed

        # Make predictions
        y_pred = model_final.predict(X_test.reshape(X_test.shape[0], -1))  # Reshape the data to 2D if needed

        # Evaluate the model
        rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        print(f'{season}:{sequence_length}:{trgt}:{features}:{rmse_test}:{r2}')
        # Write results to a CSV file
        formatted_dates = np.array(['-'.join(map(str, row)) for row in X_test])
        formatted_dates_df = pd.DataFrame(formatted_dates, columns=['Date'])
        y_test_df = pd.DataFrame(y_test, columns=['actual'])
        y_preds = pd.DataFrame(y_pred, columns=['preds'])
        results = pd.concat([formatted_dates_df, y_test_df, y_preds.reset_index(drop=True)], axis=1)
        results.to_csv(f'Results2/Preliminary_{trgt}_res{res}D_BestFeatures{sequence_length}_LGBMRegressor_{season}_results.csv', index=False)
# #PLOT
# # Assuming model_final is your trained LightGBM model
# get_lgbm_varimp(model_final, sequence_length, features)

resolution:Season:Target:Feature:RMSEtesting
Autumn:1:T0:['Tair', 'SNODP', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'EVPSOIL']:1.6364512974271255:0.9545329642938375
Autumn:1:T16:['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'TSH', 'SLP']:2.4547241138174876:0.8329569205896457
Autumn:1:T31:['T2M', 'SWLAND', 'GHTSKIN', 'TLML', 'QV2M', 'SLP']:1.7737432503970871:0.7140919236479909
Autumn:1:T46:['Tair', 'T2M', 'SWLAND', 'GHTSKIN', 'TLML', 'QV2M', 'SLP']:0.4656884548847183:0.7087466093234069
Autumn:1:T76:['Tair', 'SWGDN', 'T2M', 'SWLAND', 'GHTSKIN', 'TS']:0.08994768173619504:0.5164165137828638
Autumn:1:T97:['Tair', 'SNODP', 'LWGAB', 'GHTSKIN', 'TSH', 'LWLAND', 'SLP']:0.020939451954835916:0.007806056824202101
Autumn:4:T0:['Tair', 'SNODP', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'EVPSOIL']:2.019706677816514:0.9307245820261683
Autumn:4:T16:['Tair', 'SNODP', 'SWGDN', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'TSH', 'SLP']:2.6874445734490737:0.8003556167558348
Autumn:4:T31:['